In [ ]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau

def sir(G, s, maxitr, beta, gamma):
    sum_sp = 0  # sum of all recovered nodes in all iterations
    
    neighbor_sets = {node: set(G[node]) for node in G.nodes()}
    for i in range(maxitr):
        infected = set(s)
        recovered = set()
        while infected:
            new_infected = set()
            for node in infected:
                # Retrieve precomputed neighbor set
                neighbors = neighbor_sets[node]
                for neighbor in neighbors:
                    if random.random() < beta:
                        if neighbor not in infected and neighbor not in recovered:
                            new_infected.add(neighbor)
                if random.random() < gamma:
                    recovered.add(node)
            infected |= new_infected
            infected -= recovered
        sum_sp += len(recovered)
    
    avg_sp = sum_sp / maxitr
    return avg_sp

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo


# Define methods
def method1(G):
    return nx.degree_centrality(G)

def method2(G):
    return nx.betweenness_centrality(G)

def method3(G):
    return nx.closeness_centrality(G)

def method4(G):
    return nx.core_number(G)

def method5(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

def method6(G):
    ks = nx.core_number(G)
    cnc = method5(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def method7(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_max_set = {node for node, k in ks.items() if k == ks_max}
    teta = {}
    for vi in G.nodes():
        teta_vi = (ks_max - ks[vi] + 1)
        total_dis = 0
        for vj in ks_max_set:
            if nx.has_path(G, vi, vj):
                total_dis += nx.shortest_path_length(G, vi, vj)
        teta[vi] = teta_vi * total_dis
    return teta

def method8(G):
    ks = nx.core_number(G)
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    ks_max = max(ks.values())
    ks_norm = {node: ks[node] / ks_max for node in G.nodes()}
    all_around = {v: math.sqrt(ks_norm[v]**2 + bet[v]**2 + deg[v]**2) for v in G.nodes()}
    return all_around

def h_index_centrality(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

def method10(G):
    HI = h_index_centrality(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

def method11(G):
    nv = {}
    for vi in G.nodes():
        set1 = set()
        for vj in G.neighbors(vi):
            set1.add(vj)
            for vk in G.neighbors(vj):
                set1.add(vk)
        nv[vi] = len(set1) - 1
    
    c_local = {}
    for vi in G.nodes():
        total1 = 0
        for vj in G.neighbors(vi):
            total2 = 0
            for vk in G.neighbors(vj):
                total2 += nv[vk]
            total1 += total2
        c_local[vi] = total1
    return c_local

def method12(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

def method13(G):
    CLC = nx.clustering(G)
    CLGC = {}
    for i in G.nodes():
        CLGC[i] = sum(
            math.sqrt(CLC[j]) / nx.shortest_path_length(G, i, j)
            for j in G.nodes()
            if i != j and nx.has_path(G, i, j)
        ) * CLC[i]
    
    ECLGC = {}
    for v in G.nodes():
        sum1 = sum2 = sum3 = 0
        neighbors_v = list(G.neighbors(v))
        for u in G.nodes():
            if u != v and nx.has_path(G, v, u):
                neighbors_u = list(G.neighbors(u))
                lu = len(neighbors_u)
                sum1 += CLC[u] / len(neighbors_v)
                sum2 = sum(CLC[j] / lu for j in neighbors_u)
                sum3 += math.sqrt(CLC[u] + sum2) / nx.shortest_path_length(G, v, u)
        ECLGC[v] = sum1 * sum3
    return ECLGC

def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    order_of_magnitude = int(math.log10(size_of_V))
    alpha = size_of_V / order_of_magnitude
    return alpha

def method14(G, m=1):
    alpha = compute_alpha(G)
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes:
        cenC_i = 0
        
        # Get m-neighborhood of node i
        m_neighborhood = set(nx.single_source_shortest_path_length(G, i, cutoff=m).keys())
        m_neighborhood.remove(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij == 0:
                continue
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            term1 = (math.exp(alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = (k_s_j / (abs(k_s_i - k_s_j) + 1))
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def method15(G):
    clustering_coeffs = nx.clustering(G)
    cluster_rank_scores = {node: 0.0 for node in G.nodes()}

    degrees = dict(G.degree())

    # Calculate ClusterRank scores
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(degrees[neighbor] + 1 for neighbor in neighbors)
        cluster_rank_scores[node] = 10 ** (-clustering_coeffs[node]) * sum_term

    return cluster_rank_scores

# Optimized methods
def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())

# Improved Gravity Centrality
def method16(G):
    IGC = {}
    ks = nx.core_number(G)
    k = dict(nx.degree(G))
    for i in G.nodes():
        IGC[i] = 0
        for j in G.neighbors(i):
            neighborhood = get_neighborhood(G, j, 2)
            for p in neighborhood:
                IGC[i] += (ks[j] * k[p]) / (nx.shortest_path_length(G, j, p)**2)
    return IGC

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method17(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

def method18(G):
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    entropy_centrality = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            entropy_centrality[node] = 0
        else:
            sum_degrees = sum(G.degree(neighbor) for neighbor in neighbors)
            sum_centrality = sum(degree_centrality[neighbor] for neighbor in neighbors)
            if sum_centrality == 0:
                entropy_centrality[node] = 0
            else:
                entropy_centrality[node] = closeness_centrality[node] * (sum_degrees / sum_centrality)
    
    return entropy_centrality

# KNC
def method19(G):
    k_shell_values = nx.core_number(G)
    degree_values = dict(G.degree())
    clustering_coeff = nx.clustering(G)
    
    alpha = compute_alpha(G)
    KNC = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(k_shell_values[neighbor] * degree_values[neighbor] * clustering_coeff[neighbor] for neighbor in neighbors)
        KNC[node] = alpha * sum_term
    
    return KNC

# NEDC
def method20(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    k_shell_values = nx.core_number(G)
    
    nodes = G.nodes()
    NEDC = {}
    
    for node in nodes:
        term1 = degree_centrality[node] * betweenness_centrality[node]
        term2 = closeness_centrality[node] * k_shell_values[node]
        NEDC[node] = term1 + term2
    
    return NEDC

# ====================================================

# Method 21 (LGC)
def method21(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 22 (SEGM)
def method22(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 23 (MCGM)
def method23(G):
    k = dict(G.degree())
    ks = nx.core_number(G)
    x = nx.eigenvector_centrality(G)

    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))

    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)

    k_max = max(k_values)
    ks_max = max(ks_values)
    x_max = max(x_values)

    alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)

    MCGM = dict()
    R = 3

    for i in G.nodes():
        MCGM[i] = 0
        for j in G.nodes():
            if i != j and nx.shortest_path_length(G, source=i, target=j) <= R:
                MCGM[i] += ((k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max) *
                            (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max) /
                            (nx.shortest_path_length(G, source=i, target=j) ** 2))

    return MCGM

# Method 24 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method24(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC

# Method 25 (MDD)
def method25(G):
    def compute_mixed_degrees(H, landa):
        km = {}
        for v in H.nodes():
            kr = H.degree(v)
            ke = G.degree(v) - kr
            km[v] = kr + landa * ke
        return km

    H = G.copy()
    landa = 0.7
    rank = {}
    km = {v: 1 for v in G.nodes()}  # Initialize km values

    while H.number_of_nodes() > 0:
        km = compute_mixed_degrees(H, landa)
        min_km_value = min(km.values())

        # Find and remove nodes with mixed degree <= min_km_value
        node_set = [v for v in H.nodes() if km[v] == min_km_value]
        for each in node_set:
            rank[each] = km[each]
            H.remove_node(each)

    return rank

# Method 26 (BaseGM)
def method26(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 27 (GlobalGM)
def method27(G):
    gm = dict()
    k = dict(nx.degree(G))
    for vi in G.nodes():
        gm[vi] = 0
        for vj in G.nodes():
            if vj != vi:
                gm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gm

# Method 28 (LocalGM)
def method28(G):
    lgm = dict()
    radius = 3
    k = dict(nx.degree(G))
    for vi in G.nodes():
        lgm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                lgm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return lgm

# Method 29 (WGravity)
def method29(G):
    gmm = dict()
    ev = nx.eigenvector_centrality(G)
    k = dict(nx.degree(G))

    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        gmm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                gmm[vi] += (ev[vi] * (k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gmm


# Method 30 (HKS)
def method30(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 31 (SHKS)
def method31(G):
    def compute_efficiency(G):
        n = len(G)
        if n <= 1:
            return 0
        inv_distances = (1 / d for u, v_dict in nx.shortest_path_length(G) for v, d in v_dict.items() if d > 0)
        return sum(inv_distances) / (n * (n - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_copy = G.copy()
            G_copy.remove_node(k)
            centrality[k] = (efficiency_G - compute_efficiency(G_copy)) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    def network_constrained_coefficient(G, v):
        neighbors = list(G.neighbors(v))
        coefficient = sum(calculate_pvu(G, v, u) + sum((calculate_pvu(G, v, w) * calculate_pvu(G, w, u)) ** 2 for w in set(G.neighbors(u)) & set(G.neighbors(v))) for u in neighbors)
        return coefficient

    def calculate_pvu(G, v, u):
        return (1 if u in G.neighbors(v) else 0) / G.degree(v)

    efficiency_centrality = compute_efficiency_centrality(G)
    Constraint_Coef = {node: network_constrained_coefficient(G, node) for node in G.nodes()}
    sh = {node: 1 / Constraint_Coef[node] for node in G.nodes()}
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    C = {v: sum(I[v] + I[u] for u in G.neighbors(v)) for v in G.nodes()}
    IS = {v: sum(C[u] for u in G.neighbors(v)) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in G.neighbors(v)) for v in G.nodes()}
    return SHKS

# Method 32 (KSGC)
def method32(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    n = G.number_of_nodes()
    C = np.zeros((n, n))
    F = np.zeros((n, n))
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    for i in range(n):
        for j in range(n):
            if i != j:
                vi, vj = nodes[i], nodes[j]
                try:
                    C[i, j] = np.exp((ks[vi] - ks[vj]) / (ks_max - ks_min))
                    F[i, j] = C[i, j] * (k[vi] * k[vj] / nx.shortest_path_length(G, vi, vj) ** 2)
                except nx.NetworkXNoPath:
                    F[i, j] = 0

    KSGC = dict()
    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        vi_index = node_index[vi]
        KSGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            vj_index = node_index[vj]
            KSGC[vi] += F[vi_index, vj_index]
    
    return KSGC

# Method 33 (EFFC)
def method33(G):
    def compute_efficiency(G):
        N = len(G.nodes)
        if N < 2:
            return 0
        efficiency_sum = 0
        for i in G.nodes:
            for j in G.nodes:
                if i != j:
                    try:
                        shortest_path_length = nx.shortest_path_length(G, source=i, target=j)
                        efficiency_sum += 1 / shortest_path_length
                    except nx.NetworkXNoPath:
                        efficiency_sum += 0  # Adding 0 for disconnected pairs
        return efficiency_sum / (N * (N - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_k_prime = G.copy()
            G_k_prime.remove_node(k)
            efficiency_G_k_prime = compute_efficiency(G_k_prime)
            centrality[k] = (efficiency_G - efficiency_G_k_prime) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    return compute_efficiency_centrality(G)

# Method 34 (Local Relative ASP)
def method34(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 35 (InformationRank)
def method35(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score


# Method 36 (Weighted K-shell)
def method36(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# Method 37 (HybridKshell)
def method37(G):
    radius = 2
    ks = nx.core_number(G)
    k = dict(G.degree())
    ksh = dict()
    landa = 0.4

    for vi in G.nodes():
        ksh[vi] = 0
        neighbors = k_neighbors(G, vi, radius)
        for vj in neighbors:
            ksh[vi] += (math.sqrt(ks[vi] + ks[vj]) + landa * k[vj]) / (nx.shortest_path_length(G, vi, vj) ** 2)

    return ksh

# Optimized method 38 (Social Capital)
def method38(G):
    SC = {}
    K = dict(G.degree())
    for i in G.nodes:
        SC[i] = K[i] + sum(K[j] for j in G.neighbors(i))
    return SC

# Optimized method 39 (Potential Edge Weight)
def method39(G):
    KW = {}
    k = dict(G.degree())
    Landa = 0.5

    for vi in G.nodes:
        KW[vi] = Landa * k[vi] + sum((1 - Landa) * (k[vi] + k[vj]) for vj in G.neighbors(vi))
        KW[vi] = int(KW[vi])

    def find_and_remove_nodes(H, ks):
        nodes_to_remove = [node for node in list(H.nodes) if KW[node] <= ks]
        H.remove_nodes_from(nodes_to_remove)
        return nodes_to_remove

    H = G.copy()
    ks = min(KW.values())
    kshell = {}
    tmp = []

    while H.nodes:
        nodes_to_remove = find_and_remove_nodes(H, ks)
        if not nodes_to_remove:
            if tmp:
                kshell[ks] = tmp
            ks += 1
            tmp = []
        else:
            tmp.extend(nodes_to_remove)
        if not H.nodes:
            kshell[ks] = tmp
            break

    weightedks = {}
    wks = 1
    for ks, value in kshell.items():
        if value:
            weightedks[wks] = value
            wks += 1
            
    output_wks = {}
    for weight, nodes in weightedks.items():
        for node in nodes:
            output_wks[node] = weight

    return output_wks

# method calculate effg centrality
def method40(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality

# method 41 (IS-PEW)
def method41(G):
    import networkx as nx
    import numpy as np

    def compute_TP(G, v_A):
        # Compute NTC(v_A)
        NTC_v_A = nx.triangles(G, v_A)
        # Compute TC
        TC = sum(nx.triangles(G).values()) // 3
        return NTC_v_A / TC if TC != 0 else 0

    def compute_InfE(G, v_A, v_B):
        common_neighbors = list(nx.common_neighbors(G, v_A, v_B))
        DG_v_A = G.degree[v_A]
        DG_v_B = G.degree[v_B]
        sum_DG_k = sum(G.degree[k] for k in common_neighbors)
        return (DG_v_A * DG_v_B) / (1 + sum_DG_k)

    def compute_NIP(G, v_A):
        neighbors = list(G.neighbors(v_A))
        DG_v_a = [G.degree[v_a] for v_a in neighbors]
        KS_v_a = [nx.core_number(G)[v_a] for v_a in neighbors]
        return sum(np.sqrt(DG * KS) for DG, KS in zip(DG_v_a, KS_v_a))

    def compute_EW(G, v_A, v_B):
        TP_v_A = compute_TP(G, v_A)
        TP_v_B = compute_TP(G, v_B)
        KS_v_A = nx.core_number(G)[v_A]
        KS_v_B = nx.core_number(G)[v_B]
        NIP_v_A = compute_NIP(G, v_A)
        NIP_v_B = compute_NIP(G, v_B)
        ED_v_A_v_B = 1 / compute_InfE(G, v_A, v_B)
        return ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)

    def compute_IS(G, v_A):
        neighbors = list(G.neighbors(v_A))
        EW_values = [compute_EW(G, v_A, v_B) for v_B in neighbors]
        return sum(EW_values)

    # Main Function
    def influential_spreaders(G):
        # Step 1: Compute potential edge weight for each edge
        for u, v in G.edges():
            G[u][v]['weight'] = compute_EW(G, u, v)

        # Step 2: Identify influential spreaders
        IS_values = {v: compute_IS(G, v) for v in G.nodes()}
        return IS_values

    return influential_spreaders(G)

# k-shell iteration Factor (KS-IF)
def method42(G):
    G_copy = G.copy()
    k_shell = {}
    iteration_factors = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:
            m = iteration_nodes[-1][1]
            for node, n in iteration_nodes:
                iteration_factors[node] = k * (1 + n / m)

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    ks_IF = {}
    deg = dict(G.degree())
    for vi in G.nodes:
        ks_IF[vi] = iteration_factors[vi] * deg[vi]
        for vj in nx.neighbors(G, vi):
            ks_IF[vi] += iteration_factors[vj] * deg[vj]

    return ks_IF


# Method 43 (DKGM)
def method43(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        m = iteration_nodes[-1][1]
        for node, n in iteration_nodes:
            k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star[node]

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK[neighbor] / d_ij**2
                except nx.NetworkXNoPath:
                    continue

    return DKGM



methods = [globals()[f'method{i}'] for i in range(1, 44)]

#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc": method5,
#    "Cnc Plus": method6,
#    "Distance to Network Core": method7,
#    "All Around Node": method8,
#    "H-index Centrality": method9,
#    "Local H-index": method10,
#    "Semi-Local Centrality": method11,
#    "DNC": method12,
#    "ECLGC": method13,
#    "Centripetal Centrality": method14,
#    "Cluster Rank": method15,
#    "Improved Gravity Centrality":method16,
#    "EDBC" : method17,
#    "Entropy Centrality": method18,
#    "KNC" : method19,
#    "NEDC" : method20,
#    "LGC" : method21,
#    "SEGM" : method22,
#    "MCGM" : method23,
#    "HVGC" : method24,
#    "MDD" : method25,
#    "BaseGM" : method26,
#    "GlobalGM" : method27,
#    "LocalGM" : method28,
#    "WGravity" : method29,
#    "HKS" : method30,
#    "SHKS" : method31,
#    "KSGC" : method32,
#    "EFFC" : method33,
#    "Local Relative ASP" : method34,
#    "Information Rank" : method35,
#    "Weighted K-shell" : method36,
#    "Hybrid K-shell" : method37,
#    "Social Capital" : method38,
#    "Potential Edge Weight" : method39,
#    "effg Gravity" : method40,
#    "IS-PEW" : method41,
#    "KS-IF" : method42,
#    "DKGM" : method43
# }


# Main Code
dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25]
gamma = 1  # Adjust gamma as needed
alpha_values = [0.5, 0.6, 0.7, 0.8, 0.9]

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.csv'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)
        df = pd.read_csv(dataset_path)
        G = nx.from_pandas_edgelist(df, source='source_column', target='target_column')

        # Remove self-loops
        G.remove_edges_from(nx.selfloop_edges(G))

        for beta in beta_values:
            spread_power = {}
            for v in G.nodes():
                seed = [v]
                spread_power[v] = sir(G, seed, 1000, beta, gamma)

            results = {}
            sigma = [node for node, _ in sorted(spread_power.items(), key=lambda x: x[1], reverse=True)]

            for i, method in enumerate(methods, start=1):
                method_name = f'method{i}'
                try:
                    ranked_nodes, exec_time = measure_execution_time(method, G)
                    monotonicity = compute_monotonicity(G, ranked_nodes)
                    sir_ranks = sorted(spread_power.items(), key=lambda x: x[1], reverse=True)
                    tau, p_value = kendalltau(sir_ranks, ranked_nodes)
                    R = [node for node, _ in ranked_nodes]

                    rbo_scores = {f'RBO {alpha}': compute_rbo(sigma, R, alpha) for alpha in alpha_values}
                    
                    results[method_name] = {
                        'ranked_nodes': ranked_nodes,
                        'execution_time': exec_time,
                        'monotonicity': monotonicity,
                        f'Beta {beta}': beta,
                        f'Kendall Tau {beta}': tau,
                        f'P-Value {beta}': p_value,
                        **rbo_scores
                    }
                except Exception as e:
                    results[method_name] = {
                        'ranked_nodes': [],
                        'execution_time': None,
                        'monotonicity': None,
                        f'Beta {beta}': beta,
                        f'Kendall Tau {beta}': None,
                        f'P-Value {beta}': None,
                        **{f'RBO {alpha}': None for alpha in alpha_values},
                        'error': str(e)
                    }

            # Create DataFrame for results
            data = []
            for method_name, result in results.items():
                if isinstance(result['ranked_nodes'], list) and all(isinstance(item, tuple) and len(item) == 2 for item in result['ranked_nodes']):
                    top_5_nodes = []
                    for node, score in result['ranked_nodes'][:5]:
                        try:
                            if isinstance(score, (int, float)):  # Ensure score is a number
                                top_5_nodes.append(f"{node}:{score:.4f}")
                            else:
                                top_5_nodes.append(f"{node}:N/A")
                                print(f"Score for node {node} is not a number: {score}")
                        except (ValueError, TypeError) as e:
                            top_5_nodes.append(f"{node}:N/A")
                            print(f"Error formatting score for node {node}: {e}")
                else:
                    top_5_nodes = ['N/A'] * 5

                row = [
                    method_name, 
                    result['execution_time'], 
                    result.get('monotonicity', 'N/A'), 
                    result.get(f'Beta {beta}'), 
                    result.get(f'Kendall Tau {beta}'), 
                    result.get(f'P-Value {beta}')
                ] + [result.get(f'RBO {alpha}') for alpha in alpha_values] + top_5_nodes
                if 'error' in result:
                    row.append(result['error'])
                else:
                    row.append('')  # Ensure the row has the same number of columns
                data.append(row)

            # Ensure each row has the correct number of columns
            expected_columns = ['Method', 'Execution Time', 'Monotonicity', f'Beta {beta}', f'Kendall Tau {beta}', f'P-Value {beta}'] + [f'RBO {alpha}' for alpha in alpha_values] + [f'Top {i+1}' for i in range(5)] + ['Error']
            for row in data:
                if len(row) < len(expected_columns):
                    row.extend([''] * (len(expected_columns) - len(row)))

            result_df = pd.DataFrame(data, columns=expected_columns)

            # Write DataFrame to Excel file for each beta and dataset
            dataset_name = os.path.splitext(dataset_filename)[0]
            output_file = f'{dataset_name}_results_beta_{beta}.xlsx'
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                result_df.to_excel(writer, sheet_name='Results', index=False)

                # Save SIR spread power to Excel file
                spread_df = pd.DataFrame(list(spread_power.items()), columns=['Node', 'Spread Power'])
                spread_df.to_excel(writer, sheet_name=f'SIR_{beta}', index=False)

# Updated Code to read SIR data from datasets

In [ ]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo


# Define methods
def method1(G):
    return nx.degree_centrality(G)

def method2(G):
    return nx.betweenness_centrality(G)

def method3(G):
    return nx.closeness_centrality(G)

def method4(G):
    return nx.core_number(G)

def method5(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

def method6(G):
    ks = nx.core_number(G)
    cnc = method5(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def method7(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_max_set = {node for node, k in ks.items() if k == ks_max}
    teta = {}
    for vi in G.nodes():
        teta_vi = (ks_max - ks[vi] + 1)
        total_dis = 0
        for vj in ks_max_set:
            if nx.has_path(G, vi, vj):
                total_dis += nx.shortest_path_length(G, vi, vj)
        teta[vi] = teta_vi * total_dis
    return teta

def method8(G):
    ks = nx.core_number(G)
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    ks_max = max(ks.values())
    ks_norm = {node: ks[node] / ks_max for node in G.nodes()}
    all_around = {v: math.sqrt(ks_norm[v]**2 + bet[v]**2 + deg[v]**2) for v in G.nodes()}
    return all_around

def method9(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

def method10(G):
    HI = method9(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

def method11(G):
    nv = {}
    for vi in G.nodes():
        set1 = set()
        for vj in G.neighbors(vi):
            set1.add(vj)
            for vk in G.neighbors(vj):
                set1.add(vk)
        nv[vi] = len(set1) - 1
    
    c_local = {}
    for vi in G.nodes():
        total1 = 0
        for vj in G.neighbors(vi):
            total2 = 0
            for vk in G.neighbors(vj):
                total2 += nv[vk]
            total1 += total2
        c_local[vi] = total1
    return c_local

def method12(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

def method13(G):
    CLC = nx.clustering(G)
    CLGC = {}
    for i in G.nodes():
        CLGC[i] = sum(
            math.sqrt(CLC[j]) / nx.shortest_path_length(G, i, j)
            for j in G.nodes()
            if i != j and nx.has_path(G, i, j)
        ) * CLC[i]
    
    ECLGC = {}
    for v in G.nodes():
        sum1 = sum2 = sum3 = 0
        neighbors_v = list(G.neighbors(v))
        for u in G.nodes():
            if u != v and nx.has_path(G, v, u):
                neighbors_u = list(G.neighbors(u))
                lu = len(neighbors_u)
                sum1 += CLC[u] / len(neighbors_v)
                sum2 = sum(CLC[j] / lu for j in neighbors_u)
                sum3 += math.sqrt(CLC[u] + sum2) / nx.shortest_path_length(G, v, u)
        ECLGC[v] = sum1 * sum3
    return ECLGC

def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    order_of_magnitude = int(math.log10(size_of_V))
    alpha = size_of_V / order_of_magnitude
    return alpha

def method14(G, m=1):
    alpha = compute_alpha(G)
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes:
        cenC_i = 0
        
        # Get m-neighborhood of node i
        m_neighborhood = set(nx.single_source_shortest_path_length(G, i, cutoff=m).keys())
        m_neighborhood.remove(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij == 0:
                continue
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            term1 = (math.exp(alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = (k_s_j / (abs(k_s_i - k_s_j) + 1))
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def method15(G):
    clustering_coeffs = nx.clustering(G)
    cluster_rank_scores = {node: 0.0 for node in G.nodes()}

    degrees = dict(G.degree())

    # Calculate ClusterRank scores
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(degrees[neighbor] + 1 for neighbor in neighbors)
        cluster_rank_scores[node] = 10 ** (-clustering_coeffs[node]) * sum_term

    return cluster_rank_scores

# Optimized methods
def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())

# Improved Gravity Centrality
def method16(G):
    IGC = {}
    ks = nx.core_number(G)
    k = dict(nx.degree(G))
    for i in G.nodes():
        IGC[i] = 0
        for j in G.neighbors(i):
            neighborhood = get_neighborhood(G, j, 2)
            for p in neighborhood:
                IGC[i] += (ks[j] * k[p]) / (nx.shortest_path_length(G, j, p)**2)
    return IGC

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method17(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

def method18(G):
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    entropy_centrality = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            entropy_centrality[node] = 0
        else:
            sum_degrees = sum(G.degree(neighbor) for neighbor in neighbors)
            sum_centrality = sum(degree_centrality[neighbor] for neighbor in neighbors)
            if sum_centrality == 0:
                entropy_centrality[node] = 0
            else:
                entropy_centrality[node] = closeness_centrality[node] * (sum_degrees / sum_centrality)
    
    return entropy_centrality

# KNC
def method19(G):
    k_shell_values = nx.core_number(G)
    degree_values = dict(G.degree())
    clustering_coeff = nx.clustering(G)
    
    alpha = compute_alpha(G)
    KNC = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(k_shell_values[neighbor] * degree_values[neighbor] * clustering_coeff[neighbor] for neighbor in neighbors)
        KNC[node] = alpha * sum_term
    
    return KNC

# NEDC
def method20(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    k_shell_values = nx.core_number(G)
    
    nodes = G.nodes()
    NEDC = {}
    
    for node in nodes:
        term1 = degree_centrality[node] * betweenness_centrality[node]
        term2 = closeness_centrality[node] * k_shell_values[node]
        NEDC[node] = term1 + term2
    
    return NEDC

# ====================================================

# Method 21 (LGC)
def method21(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 22 (SEGM)
def method22(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 23 (MCGM)
def method23(G):
    k = dict(G.degree())
    ks = nx.core_number(G)
    x = nx.eigenvector_centrality(G)

    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))

    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)

    k_max = max(k_values)
    ks_max = max(ks_values)
    x_max = max(x_values)

    alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)

    MCGM = dict()
    R = 3

    for i in G.nodes():
        MCGM[i] = 0
        for j in G.nodes():
            if i != j and nx.shortest_path_length(G, source=i, target=j) <= R:
                MCGM[i] += ((k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max) *
                            (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max) /
                            (nx.shortest_path_length(G, source=i, target=j) ** 2))

    return MCGM

# Method 24 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method24(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC

# Method 25 (MDD)
def method25(G):
    def compute_mixed_degrees(H, landa):
        km = {}
        for v in H.nodes():
            kr = H.degree(v)
            ke = G.degree(v) - kr
            km[v] = kr + landa * ke
        return km

    H = G.copy()
    landa = 0.7
    rank = {}
    km = {v: 1 for v in G.nodes()}  # Initialize km values

    while H.number_of_nodes() > 0:
        km = compute_mixed_degrees(H, landa)
        min_km_value = min(km.values())

        # Find and remove nodes with mixed degree <= min_km_value
        node_set = [v for v in H.nodes() if km[v] == min_km_value]
        for each in node_set:
            rank[each] = km[each]
            H.remove_node(each)

    return rank

# Method 26 (BaseGM)
def method26(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 27 (GlobalGM)
def method27(G):
    gm = dict()
    k = dict(nx.degree(G))
    for vi in G.nodes():
        gm[vi] = 0
        for vj in G.nodes():
            if vj != vi:
                gm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gm

# Method 28 (LocalGM)
def method28(G):
    lgm = dict()
    radius = 3
    k = dict(nx.degree(G))
    for vi in G.nodes():
        lgm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                lgm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return lgm

# Method 29 (WGravity)
def method29(G):
    gmm = dict()
    ev = nx.eigenvector_centrality(G)
    k = dict(nx.degree(G))

    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        gmm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                gmm[vi] += (ev[vi] * (k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gmm


# Method 30 (HKS)
def method30(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 31 (SHKS)
def method31(G):
    def compute_efficiency(G):
        n = len(G)
        if n <= 1:
            return 0
        inv_distances = (1 / d for u, v_dict in nx.shortest_path_length(G) for v, d in v_dict.items() if d > 0)
        return sum(inv_distances) / (n * (n - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_copy = G.copy()
            G_copy.remove_node(k)
            centrality[k] = (efficiency_G - compute_efficiency(G_copy)) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    def network_constrained_coefficient(G, v):
        neighbors = list(G.neighbors(v))
        coefficient = sum(calculate_pvu(G, v, u) + sum((calculate_pvu(G, v, w) * calculate_pvu(G, w, u)) ** 2 for w in set(G.neighbors(u)) & set(G.neighbors(v))) for u in neighbors)
        return coefficient

    def calculate_pvu(G, v, u):
        return (1 if u in G.neighbors(v) else 0) / G.degree(v)

    efficiency_centrality = compute_efficiency_centrality(G)
    Constraint_Coef = {node: network_constrained_coefficient(G, node) for node in G.nodes()}
    sh = {node: 1 / Constraint_Coef[node] for node in G.nodes()}
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    C = {v: sum(I[v] + I[u] for u in G.neighbors(v)) for v in G.nodes()}
    IS = {v: sum(C[u] for u in G.neighbors(v)) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in G.neighbors(v)) for v in G.nodes()}
    return SHKS

# Method 32 (KSGC)
def method32(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    n = G.number_of_nodes()
    C = np.zeros((n, n))
    F = np.zeros((n, n))
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    for i in range(n):
        for j in range(n):
            if i != j:
                vi, vj = nodes[i], nodes[j]
                try:
                    C[i, j] = np.exp((ks[vi] - ks[vj]) / (ks_max - ks_min))
                    F[i, j] = C[i, j] * (k[vi] * k[vj] / nx.shortest_path_length(G, vi, vj) ** 2)
                except nx.NetworkXNoPath:
                    F[i, j] = 0

    KSGC = dict()
    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        vi_index = node_index[vi]
        KSGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            vj_index = node_index[vj]
            KSGC[vi] += F[vi_index, vj_index]
    
    return KSGC

# Method 33 (EFFC)
def method33(G):
    def compute_efficiency(G):
        N = len(G.nodes)
        if N < 2:
            return 0
        efficiency_sum = 0
        for i in G.nodes:
            for j in G.nodes:
                if i != j:
                    try:
                        shortest_path_length = nx.shortest_path_length(G, source=i, target=j)
                        efficiency_sum += 1 / shortest_path_length
                    except nx.NetworkXNoPath:
                        efficiency_sum += 0  # Adding 0 for disconnected pairs
        return efficiency_sum / (N * (N - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_k_prime = G.copy()
            G_k_prime.remove_node(k)
            efficiency_G_k_prime = compute_efficiency(G_k_prime)
            centrality[k] = (efficiency_G - efficiency_G_k_prime) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    return compute_efficiency_centrality(G)

# Method 34 (Local Relative ASP)
def method34(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 35 (InformationRank)
def method35(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score


# Method 36 (Weighted K-shell)
def method36(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# Method 37 (HybridKshell)
def method37(G):
    radius = 2
    ks = nx.core_number(G)
    k = dict(G.degree())
    ksh = dict()
    landa = 0.4

    for vi in G.nodes():
        ksh[vi] = 0
        neighbors = k_neighbors(G, vi, radius)
        for vj in neighbors:
            ksh[vi] += (math.sqrt(ks[vi] + ks[vj]) + landa * k[vj]) / (nx.shortest_path_length(G, vi, vj) ** 2)

    return ksh

# Optimized method 38 (Social Capital)
def method38(G):
    SC = {}
    K = dict(G.degree())
    for i in G.nodes:
        SC[i] = K[i] + sum(K[j] for j in G.neighbors(i))
    return SC

# Optimized method 39 (Potential Edge Weight)
def method39(G):
    KW = {}
    k = dict(G.degree())
    Landa = 0.5

    for vi in G.nodes:
        KW[vi] = Landa * k[vi] + sum((1 - Landa) * (k[vi] + k[vj]) for vj in G.neighbors(vi))
        KW[vi] = int(KW[vi])

    def find_and_remove_nodes(H, ks):
        nodes_to_remove = [node for node in list(H.nodes) if KW[node] <= ks]
        H.remove_nodes_from(nodes_to_remove)
        return nodes_to_remove

    H = G.copy()
    ks = min(KW.values())
    kshell = {}
    tmp = []

    while H.nodes:
        nodes_to_remove = find_and_remove_nodes(H, ks)
        if not nodes_to_remove:
            if tmp:
                kshell[ks] = tmp
            ks += 1
            tmp = []
        else:
            tmp.extend(nodes_to_remove)
        if not H.nodes:
            kshell[ks] = tmp
            break

    weightedks = {}
    wks = 1
    for ks, value in kshell.items():
        if value:
            weightedks[wks] = value
            wks += 1
            
    output_wks = {}
    for weight, nodes in weightedks.items():
        for node in nodes:
            output_wks[node] = weight

    return output_wks

# method calculate effg centrality
def method40(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality

# method 41 (IS-PEW)
def method41(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# k-shell iteration Factor (KS-IF)
def method42(G):
    G_copy = G.copy()
    k_shell = {}
    iteration_factors = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:
            m = iteration_nodes[-1][1]
            for node, n in iteration_nodes:
                iteration_factors[node] = k * (1 + n / m)

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    ks_IF = {}
    deg = dict(G.degree())
    for vi in G.nodes:
        ks_IF[vi] = iteration_factors[vi] * deg[vi]
        for vj in nx.neighbors(G, vi):
            ks_IF[vi] += iteration_factors[vj] * deg[vj]

    return ks_IF


# Method 43 (DKGM)
def method43(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        m = iteration_nodes[-1][1]
        for node, n in iteration_nodes:
            k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star[node]

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK[neighbor] / d_ij**2
                except nx.NetworkXNoPath:
                    continue

    return DKGM



methods = [globals()[f'method{i}'] for i in range(1, 44)]

#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc": method5,
#    "Cnc Plus": method6,
#    "Distance to Network Core": method7,
#    "All Around Node": method8,
#    "H-index Centrality": method9,
#    "Local H-index": method10,
#    "Semi-Local Centrality": method11,
#    "DNC": method12,
#    "ECLGC": method13,
#    "Centripetal Centrality": method14,
#    "Cluster Rank": method15,
#    "Improved Gravity Centrality":method16,
#    "EDBC" : method17,
#    "Entropy Centrality": method18,
#    "KNC" : method19,
#    "NEDC" : method20,
#    "LGC" : method21,
#    "SEGM" : method22,
#    "MCGM" : method23,
#    "HVGC" : method24,
#    "MDD" : method25,
#    "BaseGM" : method26,
#    "GlobalGM" : method27,
#    "LocalGM" : method28,
#    "WGravity" : method29,
#    "HKS" : method30,
#    "SHKS" : method31,
#    "KSGC" : method32,
#    "EFFC" : method33,
#    "Local Relative ASP" : method34,
#    "Information Rank" : method35,
#    "Weighted K-shell" : method36,
#    "Hybrid K-shell" : method37,
#    "Social Capital" : method38,
#    "Potential Edge Weight" : method39,
#    "effg Gravity" : method40,
#    "IS-PEW" : method41,
#    "KS-IF" : method42,
#    "DKGM" : method43
# }

# Main Code
dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
alpha_values = [0.5, 0.6, 0.7, 0.8, 0.9]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]  # Percentage of nodes for SI calculation

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)

        # Load edge list from "Sheet1"
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')

        # Remove self-loops
        G.remove_edges_from(nx.selfloop_edges(G))

        # Load precomputed SIR values from the "SIR" sheet
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)

        results = {}

        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_filename}. Skipping...")
                continue

            # Get SIR spread power rankings
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)

            for i, method in enumerate(methods, start=1):
                method_name = f'method{i}'
                try:
                    ranked_nodes, exec_time = measure_execution_time(method, G)
                    monotonicity = compute_monotonicity(G, ranked_nodes)
                    tau, p_value = kendalltau(
                        [spread_power[node] for node in sigma],
                        [spread_power[node] for node, _ in ranked_nodes]
                    )
                    R = [node for node, _ in ranked_nodes]

                    rbo_scores = {f'RBO {alpha}': compute_rbo(sigma, R, alpha) for alpha in alpha_values}

                    # Compute Spread Impact (SI) for f = [0.01, ..., 0.1]
                    n = len(G.nodes())  # Total number of nodes
                    si_scores = {}
                    for f in f_values:
                        top_f_nodes = [node for node, _ in ranked_nodes[:int(f * n)]]  # Top f × n nodes
                        si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * n)

                    results[(method_name, beta)] = {
                        'ranked_nodes': ranked_nodes,
                        'execution_time': exec_time,
                        'monotonicity': monotonicity,
                        f'Beta {beta}': beta,
                        f'Kendall Tau {beta}': tau,
                        f'P-Value {beta}': p_value,
                        **rbo_scores,
                        **si_scores  # Include SI values
                    }
                except Exception as e:
                    results[(method_name, beta)] = {
                        'ranked_nodes': [],
                        'execution_time': None,
                        'monotonicity': None,
                        f'Beta {beta}': beta,
                        f'Kendall Tau {beta}': None,
                        f'P-Value {beta}': None,
                        **{f'RBO {alpha}': None for alpha in alpha_values},
                        **{f'SI {f:.2f}': None for f in f_values},  # Add None for SI values in case of error
                        'error': str(e)
                    }

        # Create DataFrame for results
        data = []
        for (method_name, beta), result in results.items():
            top_5_nodes = [
                f"{node}:{score:.4f}" if isinstance(score, (int, float)) else f"{node}:N/A"
                for node, score in result['ranked_nodes'][:5]
            ] if result['ranked_nodes'] else ['N/A'] * 5

            row = [
                method_name,
                result['execution_time'],
                result.get('monotonicity', 'N/A'),
                result.get(f'Beta {beta}'),
                result.get(f'Kendall Tau {beta}'),
                result.get(f'P-Value {beta}')
            ] + [result.get(f'RBO {alpha}') for alpha in alpha_values] + \
                [result.get(f'SI {f:.2f}') for f in f_values] + top_5_nodes  # Include SI values in the output

            if 'error' in result:
                row.append(result['error'])
            else:
                row.append('')

            data.append(row)

        expected_columns = ['Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'RBO {alpha}' for alpha in alpha_values] + \
                           [f'SI {f:.2f}' for f in f_values] + [f'Top {i+1}' for i in range(5)] + ['Error']

        result_df = pd.DataFrame(data, columns=expected_columns)

        # Write results to Excel file
        dataset_name = os.path.splitext(dataset_filename)[0]
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)  # Save edge list
            df_sir.to_excel(writer, sheet_name='SIR')  # Keep original SIR values
            result_df.to_excel(writer, sheet_name='Results', index=False)  # Save computed results

        print(f"Processed {dataset_filename}, results saved in {output_file}")



In [1]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau
import igraph as ig
import leidenalg as la

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_mrr(ranked_nodes, ground_truth, k_values):
    mrr_scores = {}
    for k in k_values:
        top_k = set(ranked_nodes[:k])
        reciprocal_ranks = [1 / (ranked_nodes.index(node) + 1) for node in ground_truth if node in top_k]
        mrr_scores[f'MRR k={k}'] = sum(reciprocal_ranks) / len(ground_truth) if reciprocal_ranks else 0
    return mrr_scores

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo


# Define methods
def method1(G):
    return nx.degree_centrality(G)

def method2(G):
    return nx.betweenness_centrality(G)

def method3(G):
    return nx.closeness_centrality(G)

def method4(G):
    return nx.core_number(G)

def method5(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

def method6(G):
    ks = nx.core_number(G)
    cnc = method5(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def method7(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_max_set = {node for node, k in ks.items() if k == ks_max}
    teta = {}
    for vi in G.nodes():
        teta_vi = (ks_max - ks[vi] + 1)
        total_dis = 0
        for vj in ks_max_set:
            if nx.has_path(G, vi, vj):
                total_dis += nx.shortest_path_length(G, vi, vj)
        teta[vi] = teta_vi * total_dis
    return teta

def method8(G):
    ks = nx.core_number(G)
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    ks_max = max(ks.values())
    ks_norm = {node: ks[node] / ks_max for node in G.nodes()}
    all_around = {v: math.sqrt(ks_norm[v]**2 + bet[v]**2 + deg[v]**2) for v in G.nodes()}
    return all_around

def method9(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

def method10(G):
    HI = method9(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

def method11(G):
    nv = {}
    for vi in G.nodes():
        set1 = set()
        for vj in G.neighbors(vi):
            set1.add(vj)
            for vk in G.neighbors(vj):
                set1.add(vk)
        nv[vi] = len(set1) - 1
    
    c_local = {}
    for vi in G.nodes():
        total1 = 0
        for vj in G.neighbors(vi):
            total2 = 0
            for vk in G.neighbors(vj):
                total2 += nv[vk]
            total1 += total2
        c_local[vi] = total1
    return c_local

def method12(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

def method13(G):
    CLC = nx.clustering(G)
    CLGC = {}
    for i in G.nodes():
        CLGC[i] = sum(
            math.sqrt(CLC[j]) / nx.shortest_path_length(G, i, j)
            for j in G.nodes()
            if i != j and nx.has_path(G, i, j)
        ) * CLC[i]
    
    ECLGC = {}
    for v in G.nodes():
        sum1 = sum2 = sum3 = 0
        neighbors_v = list(G.neighbors(v))
        for u in G.nodes():
            if u != v and nx.has_path(G, v, u):
                neighbors_u = list(G.neighbors(u))
                lu = len(neighbors_u)
                sum1 += CLC[u] / len(neighbors_v)
                sum2 = sum(CLC[j] / lu for j in neighbors_u)
                sum3 += math.sqrt(CLC[u] + sum2) / nx.shortest_path_length(G, v, u)
        ECLGC[v] = sum1 * sum3
    return ECLGC

def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    order_of_magnitude = int(math.log10(size_of_V))
    alpha = size_of_V / order_of_magnitude
    return alpha

def method14(G, m=1):
    alpha = compute_alpha(G)
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes:
        cenC_i = 0
        
        # Get m-neighborhood of node i
        m_neighborhood = set(nx.single_source_shortest_path_length(G, i, cutoff=m).keys())
        m_neighborhood.remove(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij == 0:
                continue
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            term1 = (math.exp(alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = (k_s_j / (abs(k_s_i - k_s_j) + 1))
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def method15(G):
    clustering_coeffs = nx.clustering(G)
    cluster_rank_scores = {node: 0.0 for node in G.nodes()}

    degrees = dict(G.degree())

    # Calculate ClusterRank scores
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(degrees[neighbor] + 1 for neighbor in neighbors)
        cluster_rank_scores[node] = 10 ** (-clustering_coeffs[node]) * sum_term

    return cluster_rank_scores

# Optimized methods
def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())

# Improved Gravity Centrality
def method16(G):
    IGC = {}
    ks = nx.core_number(G)
    k = dict(nx.degree(G))
    for i in G.nodes():
        IGC[i] = 0
        for j in G.neighbors(i):
            neighborhood = get_neighborhood(G, j, 2)
            for p in neighborhood:
                IGC[i] += (ks[j] * k[p]) / (nx.shortest_path_length(G, j, p)**2)
    return IGC

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method17(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

def method18(G):
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    entropy_centrality = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            entropy_centrality[node] = 0
        else:
            sum_degrees = sum(G.degree(neighbor) for neighbor in neighbors)
            sum_centrality = sum(degree_centrality[neighbor] for neighbor in neighbors)
            if sum_centrality == 0:
                entropy_centrality[node] = 0
            else:
                entropy_centrality[node] = closeness_centrality[node] * (sum_degrees / sum_centrality)
    
    return entropy_centrality

# KNC
def method19(G):
    k_shell_values = nx.core_number(G)
    degree_values = dict(G.degree())
    clustering_coeff = nx.clustering(G)
    
    alpha = compute_alpha(G)
    KNC = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(k_shell_values[neighbor] * degree_values[neighbor] * clustering_coeff[neighbor] for neighbor in neighbors)
        KNC[node] = alpha * sum_term
    
    return KNC

# NEDC
def method20(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    k_shell_values = nx.core_number(G)
    
    nodes = G.nodes()
    NEDC = {}
    
    for node in nodes:
        term1 = degree_centrality[node] * betweenness_centrality[node]
        term2 = closeness_centrality[node] * k_shell_values[node]
        NEDC[node] = term1 + term2
    
    return NEDC

# ====================================================

# Method 21 (LGC)
def method21(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 22 (SEGM)
def method22(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 23 (MCGM)
def method23(G):
    k = dict(G.degree())
    ks = nx.core_number(G)
    x = nx.eigenvector_centrality(G)

    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))

    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)

    k_max = max(k_values)
    ks_max = max(ks_values)
    x_max = max(x_values)

    alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)

    MCGM = dict()
    R = 3

    for i in G.nodes():
        MCGM[i] = 0
        for j in G.nodes():
            if i != j and nx.shortest_path_length(G, source=i, target=j) <= R:
                MCGM[i] += ((k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max) *
                            (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max) /
                            (nx.shortest_path_length(G, source=i, target=j) ** 2))

    return MCGM

# Method 24 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method24(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC

# Method 25 (MDD)
def method25(G):
    def compute_mixed_degrees(H, landa):
        km = {}
        for v in H.nodes():
            kr = H.degree(v)
            ke = G.degree(v) - kr
            km[v] = kr + landa * ke
        return km

    H = G.copy()
    landa = 0.7
    rank = {}
    km = {v: 1 for v in G.nodes()}  # Initialize km values

    while H.number_of_nodes() > 0:
        km = compute_mixed_degrees(H, landa)
        min_km_value = min(km.values())

        # Find and remove nodes with mixed degree <= min_km_value
        node_set = [v for v in H.nodes() if km[v] == min_km_value]
        for each in node_set:
            rank[each] = km[each]
            H.remove_node(each)

    return rank

# Method 26 (BaseGM)
def method26(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 27 (GlobalGM)
def method27(G):
    gm = dict()
    k = dict(nx.degree(G))
    for vi in G.nodes():
        gm[vi] = 0
        for vj in G.nodes():
            if vj != vi:
                gm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gm

# Method 28 (LocalGM)
def method28(G):
    lgm = dict()
    radius = 3
    k = dict(nx.degree(G))
    for vi in G.nodes():
        lgm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                lgm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return lgm

# Method 29 (WGravity)
def method29(G):
    gmm = dict()
    ev = nx.eigenvector_centrality(G)
    k = dict(nx.degree(G))

    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        gmm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                gmm[vi] += (ev[vi] * (k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gmm


# Method 30 (HKS)
def method30(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 31 (SHKS)
def method31(G):
    def compute_efficiency(G):
        n = len(G)
        if n <= 1:
            return 0
        inv_distances = (1 / d for u, v_dict in nx.shortest_path_length(G) for v, d in v_dict.items() if d > 0)
        return sum(inv_distances) / (n * (n - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_copy = G.copy()
            G_copy.remove_node(k)
            centrality[k] = (efficiency_G - compute_efficiency(G_copy)) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    def network_constrained_coefficient(G, v):
        neighbors = list(G.neighbors(v))
        coefficient = sum(calculate_pvu(G, v, u) + sum((calculate_pvu(G, v, w) * calculate_pvu(G, w, u)) ** 2 for w in set(G.neighbors(u)) & set(G.neighbors(v))) for u in neighbors)
        return coefficient

    def calculate_pvu(G, v, u):
        return (1 if u in G.neighbors(v) else 0) / G.degree(v)

    efficiency_centrality = compute_efficiency_centrality(G)
    Constraint_Coef = {node: network_constrained_coefficient(G, node) for node in G.nodes()}
    sh = {node: 1 / Constraint_Coef[node] for node in G.nodes()}
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    C = {v: sum(I[v] + I[u] for u in G.neighbors(v)) for v in G.nodes()}
    IS = {v: sum(C[u] for u in G.neighbors(v)) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in G.neighbors(v)) for v in G.nodes()}
    return SHKS

# Method 32 (KSGC)
def method32(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    n = G.number_of_nodes()
    C = np.zeros((n, n))
    F = np.zeros((n, n))
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    for i in range(n):
        for j in range(n):
            if i != j:
                vi, vj = nodes[i], nodes[j]
                try:
                    C[i, j] = np.exp((ks[vi] - ks[vj]) / (ks_max - ks_min))
                    F[i, j] = C[i, j] * (k[vi] * k[vj] / nx.shortest_path_length(G, vi, vj) ** 2)
                except nx.NetworkXNoPath:
                    F[i, j] = 0

    KSGC = dict()
    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        vi_index = node_index[vi]
        KSGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            vj_index = node_index[vj]
            KSGC[vi] += F[vi_index, vj_index]
    
    return KSGC

# Method 33 (EFFC)
def method33(G):
    def compute_efficiency(G):
        N = len(G.nodes)
        if N < 2:
            return 0
        efficiency_sum = 0
        for i in G.nodes:
            for j in G.nodes:
                if i != j:
                    try:
                        shortest_path_length = nx.shortest_path_length(G, source=i, target=j)
                        efficiency_sum += 1 / shortest_path_length
                    except nx.NetworkXNoPath:
                        efficiency_sum += 0  # Adding 0 for disconnected pairs
        return efficiency_sum / (N * (N - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_k_prime = G.copy()
            G_k_prime.remove_node(k)
            efficiency_G_k_prime = compute_efficiency(G_k_prime)
            centrality[k] = (efficiency_G - efficiency_G_k_prime) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    return compute_efficiency_centrality(G)

# Method 34 (Local Relative ASP)
def method34(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 35 (InformationRank)
def method35(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score


# Method 36 (Weighted K-shell)
def method36(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# Method 37 (HybridKshell)
def method37(G):
    radius = 2
    ks = nx.core_number(G)
    k = dict(G.degree())
    ksh = dict()
    landa = 0.4

    for vi in G.nodes():
        ksh[vi] = 0
        neighbors = k_neighbors(G, vi, radius)
        for vj in neighbors:
            ksh[vi] += (math.sqrt(ks[vi] + ks[vj]) + landa * k[vj]) / (nx.shortest_path_length(G, vi, vj) ** 2)

    return ksh

# Optimized method 38 (Social Capital)
def method38(G):
    SC = {}
    K = dict(G.degree())
    for i in G.nodes:
        SC[i] = K[i] + sum(K[j] for j in G.neighbors(i))
    return SC

# Optimized method 39 (Potential Edge Weight)
def method39(G):
    KW = {}
    k = dict(G.degree())
    Landa = 0.5

    for vi in G.nodes:
        KW[vi] = Landa * k[vi] + sum((1 - Landa) * (k[vi] + k[vj]) for vj in G.neighbors(vi))
        KW[vi] = int(KW[vi])

    def find_and_remove_nodes(H, ks):
        nodes_to_remove = [node for node in list(H.nodes) if KW[node] <= ks]
        H.remove_nodes_from(nodes_to_remove)
        return nodes_to_remove

    H = G.copy()
    ks = min(KW.values())
    kshell = {}
    tmp = []

    while H.nodes:
        nodes_to_remove = find_and_remove_nodes(H, ks)
        if not nodes_to_remove:
            if tmp:
                kshell[ks] = tmp
            ks += 1
            tmp = []
        else:
            tmp.extend(nodes_to_remove)
        if not H.nodes:
            kshell[ks] = tmp
            break

    weightedks = {}
    wks = 1
    for ks, value in kshell.items():
        if value:
            weightedks[wks] = value
            wks += 1
            
    output_wks = {}
    for weight, nodes in weightedks.items():
        for node in nodes:
            output_wks[node] = weight

    return output_wks

# method calculate effg centrality
def method40(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality

# method 41 (IS-PEW)
def method41(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# k-shell iteration Factor (KS-IF)
def method42(G):
    G_copy = G.copy()
    k_shell = {}
    iteration_factors = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:
            m = iteration_nodes[-1][1]
            for node, n in iteration_nodes:
                iteration_factors[node] = k * (1 + n / m)

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    ks_IF = {}
    deg = dict(G.degree())
    for vi in G.nodes:
        ks_IF[vi] = iteration_factors[vi] * deg[vi]
        for vj in nx.neighbors(G, vi):
            ks_IF[vi] += iteration_factors[vj] * deg[vj]

    return ks_IF


# Method 43 (DKGM)
def method43(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        m = iteration_nodes[-1][1]
        for node, n in iteration_nodes:
            k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star[node]

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK[neighbor] / d_ij**2
                except nx.NetworkXNoPath:
                    continue

    return DKGM

#modularity_vitality
def method44(G):
    """Detect communities using Leiden algorithm and compute modularity vitality for each node."""
    
    def detect_communities_leiden(G):
        """Detect communities using the Leiden algorithm.
        
        This function converts the NetworkX graph to an iGraph graph while preserving node names,
        runs the Leiden algorithm, and returns a dictionary mapping node names to community indices.
        """
        # Convert NetworkX graph to iGraph while preserving node names.
        ig_G = ig.Graph.TupleList(G.edges(), directed=False, vertex_name_attr='name')
        partition = la.find_partition(ig_G, la.ModularityVertexPartition)
        communities = {}
        # Map each iGraph vertex back to the original NetworkX node name.
        for idx, comm in enumerate(partition):
            for v in comm:
                communities[ig_G.vs[v]["name"]] = idx
        return communities

    def modularity(G, communities):
        """Compute the modularity of graph G with the given community partition."""
        M = G.number_of_edges()
        Q = 0
        for c in set(communities.values()):
            nodes_c = [n for n in G.nodes() if communities[n] == c]
            subgraph = G.subgraph(nodes_c)
            L_c = subgraph.number_of_edges()
            k_c = sum(G.degree(n) for n in nodes_c)
            Q += (L_c / M) - (k_c / (2 * M)) ** 2
        return Q

    def compute_hi_c(G, node, communities):
        """Compute h_i,c values for a node as per Equation (10)."""
        k_i = G.degree(node)
        hi_c = {}
        for c in set(communities.values()):
            # Count links from node to neighbors in community c.
            k_i_c = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c)
            hi_c[c] = k_i_c + (k_i if communities[node] == c else 0)
        return hi_c

    def modularity_vitality(G, communities, node):
        """Compute the modularity vitality of a node using Equation (9)."""
        Q_initial = modularity(G, communities)
        if not G.has_node(node):
            return 0  

        M = G.number_of_edges()
        k_i = G.degree(node)
        c_i = communities[node]
        
        # Compute h_i,c for the given node.
        hi_c = compute_hi_c(G, node, communities)
        
        # Compute the total degree for each community.
        d_c = {c: sum(G.degree(n) for n in G.nodes() if communities[n] == c)
               for c in set(communities.values())}

        # Get nodes in the same community as node.
        community_nodes = [n for n in G.nodes() if communities[n] == c_i]
        M_internal = G.subgraph(community_nodes).number_of_edges()
        k_i_internal = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c_i)
        
        # Compute the sum term over all communities.
        sum_term = sum((d_c[c] - hi_c[c]) ** 2 for c in hi_c)
        
        # Compute updated modularity based on Equation (9).
        Q_updated = (M_internal - k_i_internal) / (M - k_i) - (1 / (4 * (M - k_i) ** 2)) * sum_term

        return Q_initial - Q_updated

    # Detect communities using the Leiden algorithm.
    communities = detect_communities_leiden(G)

    # Compute modularity vitality for each node.
    mod_vit = {node: modularity_vitality(G, communities, node) for node in G.nodes()}
    
    return mod_vit

def method45(G, lam=1.0):
    """
    Compute the Global-and-Local Centrality (GLC) of each node in graph G.
    
    The method first clusters the network using a potential-based approach,
    then selects global critical nodes from each cluster, computes local influence
    based on k-shell values, and finally calculates the overall GLC value.
    
    Parameters:
        G   : networkx.Graph
              The input graph.
        lam : float (default=1.0)
              The fraction (lambda) of nodes that must be covered by clusters.
    
    Returns:
        GLC : dict
              A dictionary mapping each node in G to its GLC centrality value.
    """
    # --- Step 1: Clustering and Global Critical Node Selection ---
    clusters = []       # List to hold all clusters
    assigned = set()    # Set of nodes that have been assigned or marked
    total_nodes = G.number_of_nodes()
    
    # Helper function: Compute the potential of a node as defined in Eq.(10)
    def compute_potential(node):
        # Potential of node = degree(node) * (sum over j in N(node) of kin(j))
        # where kin(j) = number of neighbors of j that are in {node} U N(node)
        nbrs = set(G.neighbors(node))
        target_set = nbrs.union({node})
        s = 0
        for j in nbrs:
            s += sum(1 for nbr in G.neighbors(j) if nbr in target_set)
        return G.degree(node) * s

    # Continue clustering until the number of assigned nodes reaches lam * total_nodes
    while len(assigned) < total_nodes * lam:
        # Compute potentials for all unassigned nodes.
        potentials = {}
        for node in G.nodes():
            if node not in assigned:
                potentials[node] = compute_potential(node)
        if not potentials:
            break  # All nodes have been assigned
        
        # Select seed node with maximum potential.
        seed = max(potentials, key=potentials.get)
        pcmax = potentials[seed]
        new_cluster = set([seed])
        assigned.add(seed)
        
        # Include neighbors of the seed with potential >= pcmax/2.
        for neighbor in G.neighbors(seed):
            if neighbor not in assigned and compute_potential(neighbor) >= pcmax / 2:
                new_cluster.add(neighbor)
                assigned.add(neighbor)
                
        # Expand the cluster iteratively (three degrees of influence).
        for _ in range(3):
            # Gather neighbors of the current cluster (excluding nodes already in the cluster)
            cluster_neighbors = set()
            for node in new_cluster:
                for nbr in G.neighbors(node):
                    if nbr not in new_cluster:
                        cluster_neighbors.add(nbr)
            # Process neighbors in order of increasing degree.
            cluster_neighbors = sorted(cluster_neighbors, key=lambda x: G.degree(x))
            
            added_flag = False
            for candidate in cluster_neighbors:
                # Count the number of edges from candidate to nodes in new_cluster (kin)
                kin = sum(1 for nbr in G.neighbors(candidate) if nbr in new_cluster)
                # Count the remaining edges from candidate to nodes outside new_cluster (kout)
                kout = sum(1 for nbr in G.neighbors(candidate) if nbr not in new_cluster)
                if kin >= kout:
                    new_cluster.add(candidate)
                    assigned.add(candidate)
                    added_flag = True
            if not added_flag:
                break  # No more nodes can be added in this iteration
        
        # Mark all neighbors of the new cluster as assigned to avoid re-selection.
        for node in new_cluster:
            for nbr in G.neighbors(node):
                assigned.add(nbr)
        clusters.append(new_cluster)
    
    # From each cluster, select the global critical node (node with highest degree).
    global_critical = []
    for cluster in clusters:
        if cluster:
            gc = max(cluster, key=lambda node: G.degree(node))
            global_critical.append(gc)
    
    # --- Step 2: Local Influence Calculation ---
    # Compute the k-shell (core) numbers for all nodes.
    core_numbers = nx.core_number(G)
    
    # Compute local influence LI for each node:
    # LI(node) = sum_{j in N(node)} k_shell(j)
    LI = {}
    for node in G.nodes():
        li = 0
        for nbr in G.neighbors(node):
            li += core_numbers[nbr]
        LI[node] = li

    # --- Step 3: Overall GLC Centrality Calculation ---
    # For each node v, compute:
    #   GlobalFactor(v) = sum_{u in global_critical} (LI(u) / (2 * max(d(v,u),1)))
    #   GLC(v) = LI(v) * GlobalFactor(v)
    GLC = {}
    for v in G.nodes():
        global_factor = 0
        for u in global_critical:
            try:
                d = nx.shortest_path_length(G, source=v, target=u)
            except nx.NetworkXNoPath:
                d = float('inf')
            d = max(d, 1)  # Avoid division by zero
            global_factor += LI[u] / (2 * d)
        GLC[v] = LI[v] * global_factor

    return GLC


methods = [globals()[f'method{i}'] for i in range(1, 46)]


# Main Code
dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
alpha_values = [0.5, 0.6, 0.7, 0.8, 0.9]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]  # Percentage of nodes for SI calculation

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)

        # Load edge list from "Sheet1"
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')

        # Remove self-loops
        G.remove_edges_from(nx.selfloop_edges(G))

        # Load precomputed SIR values from the "SIR" sheet
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)

        results = {}
        method_results = {}

        # Execute methods once
        for i, method in enumerate(methods, start=1):
            method_name = f'method{i}'
            try:
                ranked_nodes, exec_time = measure_execution_time(method, G)
                monotonicity = compute_monotonicity(G, ranked_nodes)
                method_results[method_name] = {
                    'ranked_nodes': ranked_nodes,
                    'execution_time': exec_time,
                    'monotonicity': monotonicity
                }
            except Exception as e:
                method_results[method_name] = {
                    'ranked_nodes': [],
                    'execution_time': None,
                    'monotonicity': None,
                    'error': str(e)
                }

        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_filename}. Skipping...")
                continue

            # Get SIR spread power rankings
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)

            for method_name, method_data in method_results.items():
                if 'error' in method_data:
                    results[(method_name, beta)] = {
                        'Beta': beta,
                        'Kendall Tau': None,
                        'P-Value': None,
                        **{f'RBO {alpha}': None for alpha in alpha_values},
                        **{f'SI {f:.2f}': None for f in f_values},
                        'error': method_data['error']
                    }
                    continue

                ranked_nodes = method_data['ranked_nodes']
                tau, p_value = kendalltau(
                    [spread_power[node] for node in sigma],
                    [spread_power[node] for node, _ in ranked_nodes]
                )
                R = [node for node, _ in ranked_nodes]
                
                rbo_scores = {f'RBO {alpha}': compute_rbo(sigma, R, alpha) for alpha in alpha_values}

                # Compute Spread Impact (SI) for f = [0.01, ..., 0.1]
                n = len(G.nodes())  # Total number of nodes
                si_scores = {}
                for f in f_values:
                    top_f_nodes = [node for node, _ in ranked_nodes[:int(f * n)]]  # Top f × n nodes
                    si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * n)

                results[(method_name, beta)] = {
                    'Beta': beta,
                    'Kendall Tau': tau,
                    'P-Value': p_value,
                    **rbo_scores,
                    **si_scores  # Include SI values
                }

        # Create DataFrame for results
        data = []
        for (method_name, beta), result in results.items():
            row = [
                method_name,
                method_results[method_name]['execution_time'],
                method_results[method_name].get('monotonicity', 'N/A'),
                result.get('Beta'),
                result.get('Kendall Tau'),
                result.get('P-Value')
            ] + [result.get(f'RBO {alpha}') for alpha in alpha_values] + \
                [result.get(f'SI {f:.2f}') for f in f_values]  # Include SI values in the output

            if 'error' in result:
                row.append(result['error'])
            else:
                row.append('')

            data.append(row)

        expected_columns = ['Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'RBO {alpha}' for alpha in alpha_values] + \
                           [f'SI {f:.2f}' for f in f_values] + ['Error']

        result_df = pd.DataFrame(data, columns=expected_columns)

        # Write results to Excel file
        dataset_name = os.path.splitext(dataset_filename)[0]
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)  # Save edge list
            df_sir.to_excel(writer, sheet_name='SIR')  # Keep original SIR values
            result_df.to_excel(writer, sheet_name='Results', index=False)  # Save computed results

        print(f"Processed {dataset_filename}, results saved in {output_file}")


Processed Dolphins(SIR).xlsx, results saved in Dolphins(SIR)_results.xlsx


# Final Code

In [3]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau
import igraph as ig
import leidenalg as la

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo

def compute_jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

# Define methods
def method1(G):
    return nx.degree_centrality(G)

def method2(G):
    return nx.betweenness_centrality(G)

def method3(G):
    return nx.closeness_centrality(G)

def method4(G):
    return nx.core_number(G)

def method5(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

def method6(G):
    ks = nx.core_number(G)
    cnc = method5(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def method7(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_max_set = {node for node, k in ks.items() if k == ks_max}
    teta = {}
    for vi in G.nodes():
        teta_vi = (ks_max - ks[vi] + 1)
        total_dis = 0
        for vj in ks_max_set:
            if nx.has_path(G, vi, vj):
                total_dis += nx.shortest_path_length(G, vi, vj)
        teta[vi] = teta_vi * total_dis
    return teta

def method8(G):
    ks = nx.core_number(G)
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    ks_max = max(ks.values())
    ks_norm = {node: ks[node] / ks_max for node in G.nodes()}
    all_around = {v: math.sqrt(ks_norm[v]**2 + bet[v]**2 + deg[v]**2) for v in G.nodes()}
    return all_around

def method9(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

def method10(G):
    HI = method9(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

def method11(G):
    nv = {}
    for vi in G.nodes():
        set1 = set()
        for vj in G.neighbors(vi):
            set1.add(vj)
            for vk in G.neighbors(vj):
                set1.add(vk)
        nv[vi] = len(set1) - 1
    
    c_local = {}
    for vi in G.nodes():
        total1 = 0
        for vj in G.neighbors(vi):
            total2 = 0
            for vk in G.neighbors(vj):
                total2 += nv[vk]
            total1 += total2
        c_local[vi] = total1
    return c_local

def method12(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

def method13(G):
    CLC = nx.clustering(G)
    CLGC = {}
    for i in G.nodes():
        CLGC[i] = sum(
            math.sqrt(CLC[j]) / nx.shortest_path_length(G, i, j)
            for j in G.nodes()
            if i != j and nx.has_path(G, i, j)
        ) * CLC[i]
    
    ECLGC = {}
    for v in G.nodes():
        sum1 = sum2 = sum3 = 0
        neighbors_v = list(G.neighbors(v))
        for u in G.nodes():
            if u != v and nx.has_path(G, v, u):
                neighbors_u = list(G.neighbors(u))
                lu = len(neighbors_u)
                sum1 += CLC[u] / len(neighbors_v)
                sum2 = sum(CLC[j] / lu for j in neighbors_u)
                sum3 += math.sqrt(CLC[u] + sum2) / nx.shortest_path_length(G, v, u)
        ECLGC[v] = sum1 * sum3
    return ECLGC

def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    order_of_magnitude = int(math.log10(size_of_V))
    alpha = size_of_V / order_of_magnitude
    return alpha

def method14(G, m=1):
    alpha = compute_alpha(G)
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes:
        cenC_i = 0
        
        # Get m-neighborhood of node i
        m_neighborhood = set(nx.single_source_shortest_path_length(G, i, cutoff=m).keys())
        m_neighborhood.remove(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij == 0:
                continue
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            term1 = (math.exp(alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = (k_s_j / (abs(k_s_i - k_s_j) + 1))
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def method15(G):
    clustering_coeffs = nx.clustering(G)
    cluster_rank_scores = {node: 0.0 for node in G.nodes()}

    degrees = dict(G.degree())

    # Calculate ClusterRank scores
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(degrees[neighbor] + 1 for neighbor in neighbors)
        cluster_rank_scores[node] = 10 ** (-clustering_coeffs[node]) * sum_term

    return cluster_rank_scores

# Optimized methods
def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())

# Improved Gravity Centrality
def method16(G):
    IGC = {}
    ks = nx.core_number(G)
    k = dict(nx.degree(G))
    for i in G.nodes():
        IGC[i] = 0
        for j in G.neighbors(i):
            neighborhood = get_neighborhood(G, j, 2)
            for p in neighborhood:
                IGC[i] += (ks[j] * k[p]) / (nx.shortest_path_length(G, j, p)**2)
    return IGC

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method17(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

def method18(G):
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    entropy_centrality = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            entropy_centrality[node] = 0
        else:
            sum_degrees = sum(G.degree(neighbor) for neighbor in neighbors)
            sum_centrality = sum(degree_centrality[neighbor] for neighbor in neighbors)
            if sum_centrality == 0:
                entropy_centrality[node] = 0
            else:
                entropy_centrality[node] = closeness_centrality[node] * (sum_degrees / sum_centrality)
    
    return entropy_centrality

# KNC
def method19(G):
    k_shell_values = nx.core_number(G)
    degree_values = dict(G.degree())
    clustering_coeff = nx.clustering(G)
    
    alpha = compute_alpha(G)
    KNC = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(k_shell_values[neighbor] * degree_values[neighbor] * clustering_coeff[neighbor] for neighbor in neighbors)
        KNC[node] = alpha * sum_term
    
    return KNC

# NEDC
def method20(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    k_shell_values = nx.core_number(G)
    
    nodes = G.nodes()
    NEDC = {}
    
    for node in nodes:
        term1 = degree_centrality[node] * betweenness_centrality[node]
        term2 = closeness_centrality[node] * k_shell_values[node]
        NEDC[node] = term1 + term2
    
    return NEDC

# ====================================================

# Method 21 (LGC)
def method21(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 22 (SEGM)
def method22(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 23 (MCGM)
def method23(G):
    k = dict(G.degree())
    ks = nx.core_number(G)
    x = nx.eigenvector_centrality(G)

    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))

    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)

    k_max = max(k_values)
    ks_max = max(ks_values)
    x_max = max(x_values)

    alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)

    MCGM = dict()
    R = 3

    for i in G.nodes():
        MCGM[i] = 0
        for j in G.nodes():
            if i != j and nx.shortest_path_length(G, source=i, target=j) <= R:
                MCGM[i] += ((k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max) *
                            (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max) /
                            (nx.shortest_path_length(G, source=i, target=j) ** 2))

    return MCGM

# Method 24 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method24(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC

# Method 25 (MDD)
def method25(G):
    def compute_mixed_degrees(H, landa):
        km = {}
        for v in H.nodes():
            kr = H.degree(v)
            ke = G.degree(v) - kr
            km[v] = kr + landa * ke
        return km

    H = G.copy()
    landa = 0.7
    rank = {}
    km = {v: 1 for v in G.nodes()}  # Initialize km values

    while H.number_of_nodes() > 0:
        km = compute_mixed_degrees(H, landa)
        min_km_value = min(km.values())

        # Find and remove nodes with mixed degree <= min_km_value
        node_set = [v for v in H.nodes() if km[v] == min_km_value]
        for each in node_set:
            rank[each] = km[each]
            H.remove_node(each)

    return rank

# Method 26 (BaseGM)
def method26(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 27 (GlobalGM)
def method27(G):
    gm = dict()
    k = dict(nx.degree(G))
    for vi in G.nodes():
        gm[vi] = 0
        for vj in G.nodes():
            if vj != vi:
                gm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gm

# Method 28 (LocalGM)
def method28(G):
    lgm = dict()
    radius = 3
    k = dict(nx.degree(G))
    for vi in G.nodes():
        lgm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                lgm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return lgm

# Method 29 (WGravity)
def method29(G):
    gmm = dict()
    ev = nx.eigenvector_centrality(G)
    k = dict(nx.degree(G))

    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        gmm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                gmm[vi] += (ev[vi] * (k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gmm


# Method 30 (HKS)
def method30(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 31 (SHKS)
def method31(G):
    def compute_efficiency(G):
        n = len(G)
        if n <= 1:
            return 0
        inv_distances = (1 / d for u, v_dict in nx.shortest_path_length(G) for v, d in v_dict.items() if d > 0)
        return sum(inv_distances) / (n * (n - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_copy = G.copy()
            G_copy.remove_node(k)
            centrality[k] = (efficiency_G - compute_efficiency(G_copy)) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    def network_constrained_coefficient(G, v):
        neighbors = list(G.neighbors(v))
        coefficient = sum(calculate_pvu(G, v, u) + sum((calculate_pvu(G, v, w) * calculate_pvu(G, w, u)) ** 2 for w in set(G.neighbors(u)) & set(G.neighbors(v))) for u in neighbors)
        return coefficient

    def calculate_pvu(G, v, u):
        return (1 if u in G.neighbors(v) else 0) / G.degree(v)

    efficiency_centrality = compute_efficiency_centrality(G)
    Constraint_Coef = {node: network_constrained_coefficient(G, node) for node in G.nodes()}
    sh = {node: 1 / Constraint_Coef[node] for node in G.nodes()}
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    C = {v: sum(I[v] + I[u] for u in G.neighbors(v)) for v in G.nodes()}
    IS = {v: sum(C[u] for u in G.neighbors(v)) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in G.neighbors(v)) for v in G.nodes()}
    return SHKS

# Method 32 (KSGC)
def method32(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    n = G.number_of_nodes()
    C = np.zeros((n, n))
    F = np.zeros((n, n))
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    for i in range(n):
        for j in range(n):
            if i != j:
                vi, vj = nodes[i], nodes[j]
                try:
                    C[i, j] = np.exp((ks[vi] - ks[vj]) / (ks_max - ks_min))
                    F[i, j] = C[i, j] * (k[vi] * k[vj] / nx.shortest_path_length(G, vi, vj) ** 2)
                except nx.NetworkXNoPath:
                    F[i, j] = 0

    KSGC = dict()
    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        vi_index = node_index[vi]
        KSGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            vj_index = node_index[vj]
            KSGC[vi] += F[vi_index, vj_index]
    
    return KSGC

# Method 33 (EFFC)
def method33(G):
    def compute_efficiency(G):
        N = len(G.nodes)
        if N < 2:
            return 0
        efficiency_sum = 0
        for i in G.nodes:
            for j in G.nodes:
                if i != j:
                    try:
                        shortest_path_length = nx.shortest_path_length(G, source=i, target=j)
                        efficiency_sum += 1 / shortest_path_length
                    except nx.NetworkXNoPath:
                        efficiency_sum += 0  # Adding 0 for disconnected pairs
        return efficiency_sum / (N * (N - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_k_prime = G.copy()
            G_k_prime.remove_node(k)
            efficiency_G_k_prime = compute_efficiency(G_k_prime)
            centrality[k] = (efficiency_G - efficiency_G_k_prime) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    return compute_efficiency_centrality(G)

# Method 34 (Local Relative ASP)
def method34(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 35 (InformationRank)
def method35(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score


# Method 36 (Weighted K-shell)
def method36(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# Method 37 (HybridKshell)
def method37(G):
    radius = 2
    ks = nx.core_number(G)
    k = dict(G.degree())
    ksh = dict()
    landa = 0.4

    for vi in G.nodes():
        ksh[vi] = 0
        neighbors = k_neighbors(G, vi, radius)
        for vj in neighbors:
            ksh[vi] += (math.sqrt(ks[vi] + ks[vj]) + landa * k[vj]) / (nx.shortest_path_length(G, vi, vj) ** 2)

    return ksh

# Optimized method 38 (Social Capital)
def method38(G):
    SC = {}
    K = dict(G.degree())
    for i in G.nodes:
        SC[i] = K[i] + sum(K[j] for j in G.neighbors(i))
    return SC

# Optimized method 39 (Potential Edge Weight)
def method39(G):
    KW = {}
    k = dict(G.degree())
    Landa = 0.5

    for vi in G.nodes:
        KW[vi] = Landa * k[vi] + sum((1 - Landa) * (k[vi] + k[vj]) for vj in G.neighbors(vi))
        KW[vi] = int(KW[vi])

    def find_and_remove_nodes(H, ks):
        nodes_to_remove = [node for node in list(H.nodes) if KW[node] <= ks]
        H.remove_nodes_from(nodes_to_remove)
        return nodes_to_remove

    H = G.copy()
    ks = min(KW.values())
    kshell = {}
    tmp = []

    while H.nodes:
        nodes_to_remove = find_and_remove_nodes(H, ks)
        if not nodes_to_remove:
            if tmp:
                kshell[ks] = tmp
            ks += 1
            tmp = []
        else:
            tmp.extend(nodes_to_remove)
        if not H.nodes:
            kshell[ks] = tmp
            break

    weightedks = {}
    wks = 1
    for ks, value in kshell.items():
        if value:
            weightedks[wks] = value
            wks += 1
            
    output_wks = {}
    for weight, nodes in weightedks.items():
        for node in nodes:
            output_wks[node] = weight

    return output_wks

# method calculate effg centrality
def method40(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality

# method 41 (IS-PEW)
def method41(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# k-shell iteration Factor (KS-IF)
def method42(G):
    G_copy = G.copy()
    k_shell = {}
    iteration_factors = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:
            m = iteration_nodes[-1][1]
            for node, n in iteration_nodes:
                iteration_factors[node] = k * (1 + n / m)

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    ks_IF = {}
    deg = dict(G.degree())
    for vi in G.nodes:
        ks_IF[vi] = iteration_factors[vi] * deg[vi]
        for vj in nx.neighbors(G, vi):
            ks_IF[vi] += iteration_factors[vj] * deg[vj]

    return ks_IF


# Method 43 (DKGM)
def method43(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        m = iteration_nodes[-1][1]
        for node, n in iteration_nodes:
            k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star[node]

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK[neighbor] / d_ij**2
                except nx.NetworkXNoPath:
                    continue

    return DKGM

#modularity_vitality
def method44(G):
    """Detect communities using Leiden algorithm and compute modularity vitality for each node."""
    
    def detect_communities_leiden(G):
        """Detect communities using the Leiden algorithm.
        
        This function converts the NetworkX graph to an iGraph graph while preserving node names,
        runs the Leiden algorithm, and returns a dictionary mapping node names to community indices.
        """
        # Convert NetworkX graph to iGraph while preserving node names.
        ig_G = ig.Graph.TupleList(G.edges(), directed=False, vertex_name_attr='name')
        partition = la.find_partition(ig_G, la.ModularityVertexPartition)
        communities = {}
        # Map each iGraph vertex back to the original NetworkX node name.
        for idx, comm in enumerate(partition):
            for v in comm:
                communities[ig_G.vs[v]["name"]] = idx
        return communities

    def modularity(G, communities):
        """Compute the modularity of graph G with the given community partition."""
        M = G.number_of_edges()
        Q = 0
        for c in set(communities.values()):
            nodes_c = [n for n in G.nodes() if communities[n] == c]
            subgraph = G.subgraph(nodes_c)
            L_c = subgraph.number_of_edges()
            k_c = sum(G.degree(n) for n in nodes_c)
            Q += (L_c / M) - (k_c / (2 * M)) ** 2
        return Q

    def compute_hi_c(G, node, communities):
        """Compute h_i,c values for a node as per Equation (10)."""
        k_i = G.degree(node)
        hi_c = {}
        for c in set(communities.values()):
            # Count links from node to neighbors in community c.
            k_i_c = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c)
            hi_c[c] = k_i_c + (k_i if communities[node] == c else 0)
        return hi_c

    def modularity_vitality(G, communities, node):
        """Compute the modularity vitality of a node using Equation (9)."""
        Q_initial = modularity(G, communities)
        if not G.has_node(node):
            return 0  

        M = G.number_of_edges()
        k_i = G.degree(node)
        c_i = communities[node]
        
        # Compute h_i,c for the given node.
        hi_c = compute_hi_c(G, node, communities)
        
        # Compute the total degree for each community.
        d_c = {c: sum(G.degree(n) for n in G.nodes() if communities[n] == c)
               for c in set(communities.values())}

        # Get nodes in the same community as node.
        community_nodes = [n for n in G.nodes() if communities[n] == c_i]
        M_internal = G.subgraph(community_nodes).number_of_edges()
        k_i_internal = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c_i)
        
        # Compute the sum term over all communities.
        sum_term = sum((d_c[c] - hi_c[c]) ** 2 for c in hi_c)
        
        # Compute updated modularity based on Equation (9).
        Q_updated = (M_internal - k_i_internal) / (M - k_i) - (1 / (4 * (M - k_i) ** 2)) * sum_term

        return Q_initial - Q_updated

    # Detect communities using the Leiden algorithm.
    communities = detect_communities_leiden(G)

    # Compute modularity vitality for each node.
    mod_vit = {node: modularity_vitality(G, communities, node) for node in G.nodes()}
    
    return mod_vit

#Global-and-Local Centrality (GLC)
def method45(G, lam=1.0):
    """
    Compute the Global-and-Local Centrality (GLC) of each node in graph G.
    
    The method first clusters the network using a potential-based approach,
    then selects global critical nodes from each cluster, computes local influence
    based on k-shell values, and finally calculates the overall GLC value.
    
    Parameters:
        G   : networkx.Graph
              The input graph.
        lam : float (default=1.0)
              The fraction (lambda) of nodes that must be covered by clusters.
    
    Returns:
        GLC : dict
              A dictionary mapping each node in G to its GLC centrality value.
    """
    # --- Step 1: Clustering and Global Critical Node Selection ---
    clusters = []       # List to hold all clusters
    assigned = set()    # Set of nodes that have been assigned or marked
    total_nodes = G.number_of_nodes()
    
    # Helper function: Compute the potential of a node as defined in Eq.(10)
    def compute_potential(node):
        # Potential of node = degree(node) * (sum over j in N(node) of kin(j))
        # where kin(j) = number of neighbors of j that are in {node} U N(node)
        nbrs = set(G.neighbors(node))
        target_set = nbrs.union({node})
        s = 0
        for j in nbrs:
            s += sum(1 for nbr in G.neighbors(j) if nbr in target_set)
        return G.degree(node) * s

    # Continue clustering until the number of assigned nodes reaches lam * total_nodes
    while len(assigned) < total_nodes * lam:
        # Compute potentials for all unassigned nodes.
        potentials = {}
        for node in G.nodes():
            if node not in assigned:
                potentials[node] = compute_potential(node)
        if not potentials:
            break  # All nodes have been assigned
        
        # Select seed node with maximum potential.
        seed = max(potentials, key=potentials.get)
        pcmax = potentials[seed]
        new_cluster = set([seed])
        assigned.add(seed)
        
        # Include neighbors of the seed with potential >= pcmax/2.
        for neighbor in G.neighbors(seed):
            if neighbor not in assigned and compute_potential(neighbor) >= pcmax / 2:
                new_cluster.add(neighbor)
                assigned.add(neighbor)
                
        # Expand the cluster iteratively (three degrees of influence).
        for _ in range(3):
            # Gather neighbors of the current cluster (excluding nodes already in the cluster)
            cluster_neighbors = set()
            for node in new_cluster:
                for nbr in G.neighbors(node):
                    if nbr not in new_cluster:
                        cluster_neighbors.add(nbr)
            # Process neighbors in order of increasing degree.
            cluster_neighbors = sorted(cluster_neighbors, key=lambda x: G.degree(x))
            
            added_flag = False
            for candidate in cluster_neighbors:
                # Count the number of edges from candidate to nodes in new_cluster (kin)
                kin = sum(1 for nbr in G.neighbors(candidate) if nbr in new_cluster)
                # Count the remaining edges from candidate to nodes outside new_cluster (kout)
                kout = sum(1 for nbr in G.neighbors(candidate) if nbr not in new_cluster)
                if kin >= kout:
                    new_cluster.add(candidate)
                    assigned.add(candidate)
                    added_flag = True
            if not added_flag:
                break  # No more nodes can be added in this iteration
        
        # Mark all neighbors of the new cluster as assigned to avoid re-selection.
        for node in new_cluster:
            for nbr in G.neighbors(node):
                assigned.add(nbr)
        clusters.append(new_cluster)
    
    # From each cluster, select the global critical node (node with highest degree).
    global_critical = []
    for cluster in clusters:
        if cluster:
            gc = max(cluster, key=lambda node: G.degree(node))
            global_critical.append(gc)
    
    # --- Step 2: Local Influence Calculation ---
    # Compute the k-shell (core) numbers for all nodes.
    core_numbers = nx.core_number(G)
    
    # Compute local influence LI for each node:
    # LI(node) = sum_{j in N(node)} k_shell(j)
    LI = {}
    for node in G.nodes():
        li = 0
        for nbr in G.neighbors(node):
            li += core_numbers[nbr]
        LI[node] = li

    # --- Step 3: Overall GLC Centrality Calculation ---
    # For each node v, compute:
    #   GlobalFactor(v) = sum_{u in global_critical} (LI(u) / (2 * max(d(v,u),1)))
    #   GLC(v) = LI(v) * GlobalFactor(v)
    GLC = {}
    for v in G.nodes():
        global_factor = 0
        for u in global_critical:
            try:
                d = nx.shortest_path_length(G, source=v, target=u)
            except nx.NetworkXNoPath:
                d = float('inf')
            d = max(d, 1)  # Avoid division by zero
            global_factor += LI[u] / (2 * d)
        GLC[v] = LI[v] * global_factor

    return GLC

#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc": method5,
#    "Cnc Plus": method6,
#    "Distance to Network Core": method7,
#    "All Around Node": method8,
#    "H-index Centrality": method9,
#    "Local H-index": method10,
#    "Semi-Local Centrality": method11,
#    "DNC": method12,
#    "ECLGC": method13,
#    "Centripetal Centrality": method14,
#    "Cluster Rank": method15,
#    "Improved Gravity Centrality":method16,
#    "EDBC" : method17,
#    "Entropy Centrality": method18,
#    "KNC" : method19,
#    "NEDC" : method20,
#    "LGC" : method21,
#    "SEGM" : method22,
#    "MCGM" : method23,
#    "HVGC" : method24,
#    "MDD" : method25,
#    "BaseGM" : method26,
#    "GlobalGM" : method27,
#    "LocalGM" : method28,
#    "WGravity" : method29,
#    "HKS" : method30,
#    "SHKS" : method31,
#    "KSGC" : method32,
#    "EFFC" : method33,
#    "Local Relative ASP" : method34,
#    "Information Rank" : method35,
#    "Weighted K-shell" : method36,
#    "Hybrid K-shell" : method37,
#    "Social Capital" : method38,
#    "Potential Edge Weight" : method39,
#    "effg Gravity" : method40,
#    "IS-PEW" : method41,
#    "KS-IF" : method42,
#    "DKGM" : method43,
#    "modularity vitality":method44,
#    "Global-and-Local Centrality (GLC)":method45
# }

methods = [globals()[f'method{i}'] for i in range(1, 46)]


# Define k values for Jaccard similarity
k_factors = [0.01, 0.03, 0.05, 0.07, 0.085, 0.10]  # Percentages of top nodes to consider

dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
alpha_values = [0.5, 0.6, 0.7, 0.8, 0.9]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]


for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')
        G.remove_edges_from(nx.selfloop_edges(G))
        
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)
        
        num_nodes = len(G.nodes())  # Get network size
        
        results = {}
        method_results = {}
        
        for i, method in enumerate(methods, start=1):
            method_name = f'method{i}'
            try:
                ranked_nodes, exec_time = measure_execution_time(method, G)
                monotonicity = compute_monotonicity(G, ranked_nodes)
                method_results[method_name] = {
                    'ranked_nodes': ranked_nodes,
                    'execution_time': exec_time,
                    'monotonicity': monotonicity
                }
            except Exception as e:
                method_results[method_name] = {
                    'ranked_nodes': [],
                    'execution_time': None,
                    'monotonicity': None,
                    'error': str(e)
                }
        
        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_filename}. Skipping...")
                continue
            
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)
            
            for method_name, method_data in method_results.items():
                if 'error' in method_data:
                    results[(method_name, beta)] = {'Beta': beta, 'Kendall Tau': None, 'P-Value': None}
                    continue
                
                ranked_nodes = method_data['ranked_nodes']
                tau, p_value = kendalltau(
                    [spread_power[node] for node in sigma],
                    [spread_power[node] for node, _ in ranked_nodes]
                )
                
                R = [node for node, _ in ranked_nodes]
                jaccard_scores = {}
                si_scores = {}
                rbo_scores = {}
                
                for k_factor in k_factors:
                    k = max(1, int(k_factor * num_nodes))
                    top_k_sigma = set(sigma[:k])
                    top_k_R = set(R[:k])
                    jaccard_scores[f'Jaccard k={k}'] = compute_jaccard_similarity(top_k_sigma, top_k_R)
                
                for f in f_values:
                    f_count = max(1, int(f * num_nodes))
                    top_f_nodes = [node for node, _ in ranked_nodes][:f_count]
                    si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * num_nodes)
                    rbo_scores[f'RBO {f:.2f}'] = compute_rbo(sigma[:f_count], R[:f_count])
                
                results[(method_name, beta)] = {
                    'Beta': beta,
                    'Kendall Tau': tau,
                    'P-Value': p_value,
                    **jaccard_scores,
                    **si_scores,
                    **rbo_scores
                }
        
        data = []
        for (method_name, beta), result in results.items():
            row = [
                method_name,
                method_results[method_name]['execution_time'],
                method_results[method_name].get('monotonicity', 'N/A'),
                result.get('Beta'),
                result.get('Kendall Tau'),
                result.get('P-Value')
            ] + [result.get(f'Jaccard k={int(k_factor * num_nodes)}') for k_factor in k_factors] + \
              [result.get(f'SI {f:.2f}') for f in f_values] + \
              [result.get(f'RBO {f:.2f}') for f in f_values]
            
            data.append(row)
        
        expected_columns = ['Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'Jaccard k={int(k_factor * num_nodes)}' for k_factor in k_factors] + \
                           [f'SI {f:.2f}' for f in f_values] + \
                           [f'RBO {f:.2f}' for f in f_values]
        
        result_df = pd.DataFrame(data, columns=expected_columns)
        
        dataset_name = os.path.splitext(dataset_filename)[0]
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)
            df_sir.to_excel(writer, sheet_name='SIR')
            result_df.to_excel(writer, sheet_name='Results', index=False)
        
        print(f"Processed {dataset_filename}, results saved in {output_file}")


Processed Contiguous(SIR).xlsx, results saved in Contiguous(SIR)_results.xlsx


In [7]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau
import igraph as ig
import leidenalg as la

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_mrr(ground_truth, ranked_list, top_ratios):
    mrr_scores = {}
    num_nodes = len(ground_truth)
    
    for ratio in top_ratios:
        k = max(1, int(ratio * num_nodes))
        reciprocal_ranks = []
        
        for node in ground_truth[:k]:
            if node in ranked_list:
                rank = ranked_list.index(node) + 1
                reciprocal_ranks.append(1 / rank)
        
        mrr_scores[f'MRR {ratio:.2f}'] = sum(reciprocal_ranks) / k if reciprocal_ranks else 0
    
    return mrr_scores


def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo

def compute_jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

# Define methods
def method1(G):
    return nx.degree_centrality(G)

def method2(G):
    return nx.betweenness_centrality(G)

def method3(G):
    return nx.closeness_centrality(G)

def method4(G):
    return nx.core_number(G)

def method5(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

def method6(G):
    ks = nx.core_number(G)
    cnc = method5(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def method7(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_max_set = {node for node, k in ks.items() if k == ks_max}
    teta = {}
    for vi in G.nodes():
        teta_vi = (ks_max - ks[vi] + 1)
        total_dis = 0
        for vj in ks_max_set:
            if nx.has_path(G, vi, vj):
                total_dis += nx.shortest_path_length(G, vi, vj)
        teta[vi] = teta_vi * total_dis
    return teta

def method8(G):
    ks = nx.core_number(G)
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    ks_max = max(ks.values())
    ks_norm = {node: ks[node] / ks_max for node in G.nodes()}
    all_around = {v: math.sqrt(ks_norm[v]**2 + bet[v]**2 + deg[v]**2) for v in G.nodes()}
    return all_around

def method9(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

def method10(G):
    HI = method9(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

def method11(G):
    nv = {}
    for vi in G.nodes():
        set1 = set()
        for vj in G.neighbors(vi):
            set1.add(vj)
            for vk in G.neighbors(vj):
                set1.add(vk)
        nv[vi] = len(set1) - 1
    
    c_local = {}
    for vi in G.nodes():
        total1 = 0
        for vj in G.neighbors(vi):
            total2 = 0
            for vk in G.neighbors(vj):
                total2 += nv[vk]
            total1 += total2
        c_local[vi] = total1
    return c_local

def method12(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

def method13(G):
    CLC = nx.clustering(G)
    CLGC = {}
    for i in G.nodes():
        CLGC[i] = sum(
            math.sqrt(CLC[j]) / nx.shortest_path_length(G, i, j)
            for j in G.nodes()
            if i != j and nx.has_path(G, i, j)
        ) * CLC[i]
    
    ECLGC = {}
    for v in G.nodes():
        sum1 = sum2 = sum3 = 0
        neighbors_v = list(G.neighbors(v))
        for u in G.nodes():
            if u != v and nx.has_path(G, v, u):
                neighbors_u = list(G.neighbors(u))
                lu = len(neighbors_u)
                sum1 += CLC[u] / len(neighbors_v)
                sum2 = sum(CLC[j] / lu for j in neighbors_u)
                sum3 += math.sqrt(CLC[u] + sum2) / nx.shortest_path_length(G, v, u)
        ECLGC[v] = sum1 * sum3
    return ECLGC

def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    order_of_magnitude = int(math.log10(size_of_V))
    alpha = size_of_V / order_of_magnitude
    return alpha

def method14(G, m=1):
    alpha = compute_alpha(G)
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes:
        cenC_i = 0
        
        # Get m-neighborhood of node i
        m_neighborhood = set(nx.single_source_shortest_path_length(G, i, cutoff=m).keys())
        m_neighborhood.remove(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij == 0:
                continue
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            term1 = (math.exp(alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = (k_s_j / (abs(k_s_i - k_s_j) + 1))
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def method15(G):
    clustering_coeffs = nx.clustering(G)
    cluster_rank_scores = {node: 0.0 for node in G.nodes()}

    degrees = dict(G.degree())

    # Calculate ClusterRank scores
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(degrees[neighbor] + 1 for neighbor in neighbors)
        cluster_rank_scores[node] = 10 ** (-clustering_coeffs[node]) * sum_term

    return cluster_rank_scores

# Optimized methods
def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())

# Improved Gravity Centrality
def method16(G):
    IGC = {}
    ks = nx.core_number(G)
    k = dict(nx.degree(G))
    for i in G.nodes():
        IGC[i] = 0
        for j in G.neighbors(i):
            neighborhood = get_neighborhood(G, j, 2)
            for p in neighborhood:
                IGC[i] += (ks[j] * k[p]) / (nx.shortest_path_length(G, j, p)**2)
    return IGC

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method17(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

def method18(G):
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    entropy_centrality = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            entropy_centrality[node] = 0
        else:
            sum_degrees = sum(G.degree(neighbor) for neighbor in neighbors)
            sum_centrality = sum(degree_centrality[neighbor] for neighbor in neighbors)
            if sum_centrality == 0:
                entropy_centrality[node] = 0
            else:
                entropy_centrality[node] = closeness_centrality[node] * (sum_degrees / sum_centrality)
    
    return entropy_centrality

# KNC
def method19(G):
    k_shell_values = nx.core_number(G)
    degree_values = dict(G.degree())
    clustering_coeff = nx.clustering(G)
    
    alpha = compute_alpha(G)
    KNC = {}
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        sum_term = sum(k_shell_values[neighbor] * degree_values[neighbor] * clustering_coeff[neighbor] for neighbor in neighbors)
        KNC[node] = alpha * sum_term
    
    return KNC

# NEDC
def method20(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    k_shell_values = nx.core_number(G)
    
    nodes = G.nodes()
    NEDC = {}
    
    for node in nodes:
        term1 = degree_centrality[node] * betweenness_centrality[node]
        term2 = closeness_centrality[node] * k_shell_values[node]
        NEDC[node] = term1 + term2
    
    return NEDC

# ====================================================

# Method 21 (LGC)
def method21(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 22 (SEGM)
def method22(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 23 (MCGM)
def method23(G):
    k = dict(G.degree())
    ks = nx.core_number(G)
    x = nx.eigenvector_centrality(G)

    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))

    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)

    k_max = max(k_values)
    ks_max = max(ks_values)
    x_max = max(x_values)

    alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)

    MCGM = dict()
    R = 3

    for i in G.nodes():
        MCGM[i] = 0
        for j in G.nodes():
            if i != j and nx.shortest_path_length(G, source=i, target=j) <= R:
                MCGM[i] += ((k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max) *
                            (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max) /
                            (nx.shortest_path_length(G, source=i, target=j) ** 2))

    return MCGM

# Method 24 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method24(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC

# Method 25 (MDD)
def method25(G):
    def compute_mixed_degrees(H, landa):
        km = {}
        for v in H.nodes():
            kr = H.degree(v)
            ke = G.degree(v) - kr
            km[v] = kr + landa * ke
        return km

    H = G.copy()
    landa = 0.7
    rank = {}
    km = {v: 1 for v in G.nodes()}  # Initialize km values

    while H.number_of_nodes() > 0:
        km = compute_mixed_degrees(H, landa)
        min_km_value = min(km.values())

        # Find and remove nodes with mixed degree <= min_km_value
        node_set = [v for v in H.nodes() if km[v] == min_km_value]
        for each in node_set:
            rank[each] = km[each]
            H.remove_node(each)

    return rank

# Method 26 (BaseGM)
def method26(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 27 (GlobalGM)
def method27(G):
    gm = dict()
    k = dict(nx.degree(G))
    for vi in G.nodes():
        gm[vi] = 0
        for vj in G.nodes():
            if vj != vi:
                gm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gm

# Method 28 (LocalGM)
def method28(G):
    lgm = dict()
    radius = 3
    k = dict(nx.degree(G))
    for vi in G.nodes():
        lgm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                lgm[vi] += ((k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return lgm

# Method 29 (WGravity)
def method29(G):
    gmm = dict()
    ev = nx.eigenvector_centrality(G)
    k = dict(nx.degree(G))

    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        gmm[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            if vj != vi:
                gmm[vi] += (ev[vi] * (k[vi] * k[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)
    return gmm


# Method 30 (HKS)
def method30(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 31 (SHKS)
def method31(G):
    def compute_efficiency(G):
        n = len(G)
        if n <= 1:
            return 0
        inv_distances = (1 / d for u, v_dict in nx.shortest_path_length(G) for v, d in v_dict.items() if d > 0)
        return sum(inv_distances) / (n * (n - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_copy = G.copy()
            G_copy.remove_node(k)
            centrality[k] = (efficiency_G - compute_efficiency(G_copy)) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    def network_constrained_coefficient(G, v):
        neighbors = list(G.neighbors(v))
        coefficient = sum(calculate_pvu(G, v, u) + sum((calculate_pvu(G, v, w) * calculate_pvu(G, w, u)) ** 2 for w in set(G.neighbors(u)) & set(G.neighbors(v))) for u in neighbors)
        return coefficient

    def calculate_pvu(G, v, u):
        return (1 if u in G.neighbors(v) else 0) / G.degree(v)

    efficiency_centrality = compute_efficiency_centrality(G)
    Constraint_Coef = {node: network_constrained_coefficient(G, node) for node in G.nodes()}
    sh = {node: 1 / Constraint_Coef[node] for node in G.nodes()}
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    C = {v: sum(I[v] + I[u] for u in G.neighbors(v)) for v in G.nodes()}
    IS = {v: sum(C[u] for u in G.neighbors(v)) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in G.neighbors(v)) for v in G.nodes()}
    return SHKS

# Method 32 (KSGC)
def method32(G):
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    n = G.number_of_nodes()
    C = np.zeros((n, n))
    F = np.zeros((n, n))
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    for i in range(n):
        for j in range(n):
            if i != j:
                vi, vj = nodes[i], nodes[j]
                try:
                    C[i, j] = np.exp((ks[vi] - ks[vj]) / (ks_max - ks_min))
                    F[i, j] = C[i, j] * (k[vi] * k[vj] / nx.shortest_path_length(G, vi, vj) ** 2)
                except nx.NetworkXNoPath:
                    F[i, j] = 0

    KSGC = dict()
    radius = int(0.5 * nx.diameter(G))
    for vi in G.nodes():
        vi_index = node_index[vi]
        KSGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            vj_index = node_index[vj]
            KSGC[vi] += F[vi_index, vj_index]
    
    return KSGC

# Method 33 (EFFC)
def method33(G):
    def compute_efficiency(G):
        N = len(G.nodes)
        if N < 2:
            return 0
        efficiency_sum = 0
        for i in G.nodes:
            for j in G.nodes:
                if i != j:
                    try:
                        shortest_path_length = nx.shortest_path_length(G, source=i, target=j)
                        efficiency_sum += 1 / shortest_path_length
                    except nx.NetworkXNoPath:
                        efficiency_sum += 0  # Adding 0 for disconnected pairs
        return efficiency_sum / (N * (N - 1))

    def compute_efficiency_centrality(G):
        efficiency_G = compute_efficiency(G)
        centrality = {}
        for k in G.nodes:
            G_k_prime = G.copy()
            G_k_prime.remove_node(k)
            efficiency_G_k_prime = compute_efficiency(G_k_prime)
            centrality[k] = (efficiency_G - efficiency_G_k_prime) / efficiency_G if efficiency_G > 0 else 0
        return centrality

    return compute_efficiency_centrality(G)

# Method 34 (Local Relative ASP)
def method34(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 35 (InformationRank)
def method35(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score


# Method 36 (Weighted K-shell)
def method36(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# Method 37 (HybridKshell)
def method37(G):
    radius = 2
    ks = nx.core_number(G)
    k = dict(G.degree())
    ksh = dict()
    landa = 0.4

    for vi in G.nodes():
        ksh[vi] = 0
        neighbors = k_neighbors(G, vi, radius)
        for vj in neighbors:
            ksh[vi] += (math.sqrt(ks[vi] + ks[vj]) + landa * k[vj]) / (nx.shortest_path_length(G, vi, vj) ** 2)

    return ksh

# Optimized method 38 (Social Capital)
def method38(G):
    SC = {}
    K = dict(G.degree())
    for i in G.nodes:
        SC[i] = K[i] + sum(K[j] for j in G.neighbors(i))
    return SC

# Optimized method 39 (Potential Edge Weight)
def method39(G):
    KW = {}
    k = dict(G.degree())
    Landa = 0.5

    for vi in G.nodes:
        KW[vi] = Landa * k[vi] + sum((1 - Landa) * (k[vi] + k[vj]) for vj in G.neighbors(vi))
        KW[vi] = int(KW[vi])

    def find_and_remove_nodes(H, ks):
        nodes_to_remove = [node for node in list(H.nodes) if KW[node] <= ks]
        H.remove_nodes_from(nodes_to_remove)
        return nodes_to_remove

    H = G.copy()
    ks = min(KW.values())
    kshell = {}
    tmp = []

    while H.nodes:
        nodes_to_remove = find_and_remove_nodes(H, ks)
        if not nodes_to_remove:
            if tmp:
                kshell[ks] = tmp
            ks += 1
            tmp = []
        else:
            tmp.extend(nodes_to_remove)
        if not H.nodes:
            kshell[ks] = tmp
            break

    weightedks = {}
    wks = 1
    for ks, value in kshell.items():
        if value:
            weightedks[wks] = value
            wks += 1
            
    output_wks = {}
    for weight, nodes in weightedks.items():
        for node in nodes:
            output_wks[node] = weight

    return output_wks

# method calculate effg centrality
def method40(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality

# method 41 (IS-PEW)
def method41(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# k-shell iteration Factor (KS-IF)
def method42(G):
    G_copy = G.copy()
    k_shell = {}
    iteration_factors = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:
            m = iteration_nodes[-1][1]
            for node, n in iteration_nodes:
                iteration_factors[node] = k * (1 + n / m)

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    ks_IF = {}
    deg = dict(G.degree())
    for vi in G.nodes:
        ks_IF[vi] = iteration_factors[vi] * deg[vi]
        for vj in nx.neighbors(G, vi):
            ks_IF[vi] += iteration_factors[vj] * deg[vj]

    return ks_IF


# Method 43 (DKGM)
def method43(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        m = iteration_nodes[-1][1]
        for node, n in iteration_nodes:
            k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star[node]

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK[neighbor] / d_ij**2
                except nx.NetworkXNoPath:
                    continue

    return DKGM

#modularity_vitality
def method44(G):
    """Detect communities using Leiden algorithm and compute modularity vitality for each node."""
    
    def detect_communities_leiden(G):
        """Detect communities using the Leiden algorithm.
        
        This function converts the NetworkX graph to an iGraph graph while preserving node names,
        runs the Leiden algorithm, and returns a dictionary mapping node names to community indices.
        """
        # Convert NetworkX graph to iGraph while preserving node names.
        ig_G = ig.Graph.TupleList(G.edges(), directed=False, vertex_name_attr='name')
        partition = la.find_partition(ig_G, la.ModularityVertexPartition)
        communities = {}
        # Map each iGraph vertex back to the original NetworkX node name.
        for idx, comm in enumerate(partition):
            for v in comm:
                communities[ig_G.vs[v]["name"]] = idx
        return communities

    def modularity(G, communities):
        """Compute the modularity of graph G with the given community partition."""
        M = G.number_of_edges()
        Q = 0
        for c in set(communities.values()):
            nodes_c = [n for n in G.nodes() if communities[n] == c]
            subgraph = G.subgraph(nodes_c)
            L_c = subgraph.number_of_edges()
            k_c = sum(G.degree(n) for n in nodes_c)
            Q += (L_c / M) - (k_c / (2 * M)) ** 2
        return Q

    def compute_hi_c(G, node, communities):
        """Compute h_i,c values for a node as per Equation (10)."""
        k_i = G.degree(node)
        hi_c = {}
        for c in set(communities.values()):
            # Count links from node to neighbors in community c.
            k_i_c = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c)
            hi_c[c] = k_i_c + (k_i if communities[node] == c else 0)
        return hi_c

    def modularity_vitality(G, communities, node):
        """Compute the modularity vitality of a node using Equation (9)."""
        Q_initial = modularity(G, communities)
        if not G.has_node(node):
            return 0  

        M = G.number_of_edges()
        k_i = G.degree(node)
        c_i = communities[node]
        
        # Compute h_i,c for the given node.
        hi_c = compute_hi_c(G, node, communities)
        
        # Compute the total degree for each community.
        d_c = {c: sum(G.degree(n) for n in G.nodes() if communities[n] == c)
               for c in set(communities.values())}

        # Get nodes in the same community as node.
        community_nodes = [n for n in G.nodes() if communities[n] == c_i]
        M_internal = G.subgraph(community_nodes).number_of_edges()
        k_i_internal = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c_i)
        
        # Compute the sum term over all communities.
        sum_term = sum((d_c[c] - hi_c[c]) ** 2 for c in hi_c)
        
        # Compute updated modularity based on Equation (9).
        Q_updated = (M_internal - k_i_internal) / (M - k_i) - (1 / (4 * (M - k_i) ** 2)) * sum_term

        return Q_initial - Q_updated

    # Detect communities using the Leiden algorithm.
    communities = detect_communities_leiden(G)

    # Compute modularity vitality for each node.
    mod_vit = {node: modularity_vitality(G, communities, node) for node in G.nodes()}
    
    return mod_vit

#Global-and-Local Centrality (GLC)
def method45(G, lam=1.0):
    """
    Compute the Global-and-Local Centrality (GLC) of each node in graph G.
    
    The method first clusters the network using a potential-based approach,
    then selects global critical nodes from each cluster, computes local influence
    based on k-shell values, and finally calculates the overall GLC value.
    
    Parameters:
        G   : networkx.Graph
              The input graph.
        lam : float (default=1.0)
              The fraction (lambda) of nodes that must be covered by clusters.
    
    Returns:
        GLC : dict
              A dictionary mapping each node in G to its GLC centrality value.
    """
    # --- Step 1: Clustering and Global Critical Node Selection ---
    clusters = []       # List to hold all clusters
    assigned = set()    # Set of nodes that have been assigned or marked
    total_nodes = G.number_of_nodes()
    
    # Helper function: Compute the potential of a node as defined in Eq.(10)
    def compute_potential(node):
        # Potential of node = degree(node) * (sum over j in N(node) of kin(j))
        # where kin(j) = number of neighbors of j that are in {node} U N(node)
        nbrs = set(G.neighbors(node))
        target_set = nbrs.union({node})
        s = 0
        for j in nbrs:
            s += sum(1 for nbr in G.neighbors(j) if nbr in target_set)
        return G.degree(node) * s

    # Continue clustering until the number of assigned nodes reaches lam * total_nodes
    while len(assigned) < total_nodes * lam:
        # Compute potentials for all unassigned nodes.
        potentials = {}
        for node in G.nodes():
            if node not in assigned:
                potentials[node] = compute_potential(node)
        if not potentials:
            break  # All nodes have been assigned
        
        # Select seed node with maximum potential.
        seed = max(potentials, key=potentials.get)
        pcmax = potentials[seed]
        new_cluster = set([seed])
        assigned.add(seed)
        
        # Include neighbors of the seed with potential >= pcmax/2.
        for neighbor in G.neighbors(seed):
            if neighbor not in assigned and compute_potential(neighbor) >= pcmax / 2:
                new_cluster.add(neighbor)
                assigned.add(neighbor)
                
        # Expand the cluster iteratively (three degrees of influence).
        for _ in range(3):
            # Gather neighbors of the current cluster (excluding nodes already in the cluster)
            cluster_neighbors = set()
            for node in new_cluster:
                for nbr in G.neighbors(node):
                    if nbr not in new_cluster:
                        cluster_neighbors.add(nbr)
            # Process neighbors in order of increasing degree.
            cluster_neighbors = sorted(cluster_neighbors, key=lambda x: G.degree(x))
            
            added_flag = False
            for candidate in cluster_neighbors:
                # Count the number of edges from candidate to nodes in new_cluster (kin)
                kin = sum(1 for nbr in G.neighbors(candidate) if nbr in new_cluster)
                # Count the remaining edges from candidate to nodes outside new_cluster (kout)
                kout = sum(1 for nbr in G.neighbors(candidate) if nbr not in new_cluster)
                if kin >= kout:
                    new_cluster.add(candidate)
                    assigned.add(candidate)
                    added_flag = True
            if not added_flag:
                break  # No more nodes can be added in this iteration
        
        # Mark all neighbors of the new cluster as assigned to avoid re-selection.
        for node in new_cluster:
            for nbr in G.neighbors(node):
                assigned.add(nbr)
        clusters.append(new_cluster)
    
    # From each cluster, select the global critical node (node with highest degree).
    global_critical = []
    for cluster in clusters:
        if cluster:
            gc = max(cluster, key=lambda node: G.degree(node))
            global_critical.append(gc)
    
    # --- Step 2: Local Influence Calculation ---
    # Compute the k-shell (core) numbers for all nodes.
    core_numbers = nx.core_number(G)
    
    # Compute local influence LI for each node:
    # LI(node) = sum_{j in N(node)} k_shell(j)
    LI = {}
    for node in G.nodes():
        li = 0
        for nbr in G.neighbors(node):
            li += core_numbers[nbr]
        LI[node] = li

    # --- Step 3: Overall GLC Centrality Calculation ---
    # For each node v, compute:
    #   GlobalFactor(v) = sum_{u in global_critical} (LI(u) / (2 * max(d(v,u),1)))
    #   GLC(v) = LI(v) * GlobalFactor(v)
    GLC = {}
    for v in G.nodes():
        global_factor = 0
        for u in global_critical:
            try:
                d = nx.shortest_path_length(G, source=v, target=u)
            except nx.NetworkXNoPath:
                d = float('inf')
            d = max(d, 1)  # Avoid division by zero
            global_factor += LI[u] / (2 * d)
        GLC[v] = LI[v] * global_factor

    return GLC

#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc": method5,
#    "Cnc Plus": method6,
#    "Distance to Network Core": method7,
#    "All Around Node": method8,
#    "H-index Centrality": method9,
#    "Local H-index": method10,
#    "Semi-Local Centrality": method11,
#    "DNC": method12,
#    "ECLGC": method13,
#    "Centripetal Centrality": method14,
#    "Cluster Rank": method15,
#    "Improved Gravity Centrality":method16,
#    "EDBC" : method17,
#    "Entropy Centrality": method18,
#    "KNC" : method19,
#    "NEDC" : method20,
#    "LGC" : method21,
#    "SEGM" : method22,
#    "MCGM" : method23,
#    "HVGC" : method24,
#    "MDD" : method25,
#    "BaseGM" : method26,
#    "GlobalGM" : method27,
#    "LocalGM" : method28,
#    "WGravity" : method29,
#    "HKS" : method30,
#    "SHKS" : method31,
#    "KSGC" : method32,
#    "EFFC" : method33,
#    "Local Relative ASP" : method34,
#    "Information Rank" : method35,
#    "Weighted K-shell" : method36,
#    "Hybrid K-shell" : method37,
#    "Social Capital" : method38,
#    "Potential Edge Weight" : method39,
#    "effg Gravity" : method40,
#    "IS-PEW" : method41,
#    "KS-IF" : method42,
#    "DKGM" : method43,
#    "modularity vitality":method44,
#    "Global-and-Local Centrality (GLC)":method45
# }

methods = [globals()[f'method{i}'] for i in range(1, 46)]

dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
k_factors = [0.01, 0.03, 0.05, 0.07, 0.085, 0.10]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')
        G.remove_edges_from(nx.selfloop_edges(G))
        
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)
        
        num_nodes = len(G.nodes())
        results = {}
        method_results = {}
        
        for i, method in enumerate(methods, start=1):
            method_name = f'method{i}'
            try:
                ranked_nodes, exec_time = measure_execution_time(method, G)
                monotonicity = compute_monotonicity(G, ranked_nodes)
                method_results[method_name] = {
                    'ranked_nodes': ranked_nodes,
                    'execution_time': exec_time,
                    'monotonicity': monotonicity
                }
            except Exception as e:
                method_results[method_name] = {
                    'ranked_nodes': [],
                    'execution_time': None,
                    'monotonicity': None,
                    'error': str(e)
                }
        
        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_filename}. Skipping...")
                continue
            
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)
            
            for method_name, method_data in method_results.items():
                if 'error' in method_data:
                    results[(method_name, beta)] = {'Beta': beta, 'Kendall Tau': None, 'P-Value': None}
                    continue
                
                ranked_nodes = method_data['ranked_nodes']
                tau, p_value = kendalltau(
                    [spread_power[node] for node in sigma],
                    [spread_power[node] for node, _ in ranked_nodes]
                )
                
                R = [node for node, _ in ranked_nodes]
                jaccard_scores = {}
                si_scores = {}
                rbo_scores = {}
                mrr_scores = compute_mrr(sigma, R, k_factors)
                
                for k_factor in k_factors:
                    k = max(1, int(k_factor * num_nodes))
                    top_k_sigma = set(sigma[:k])
                    top_k_R = set(R[:k])
                    jaccard_scores[f'Jaccard k={k}'] = compute_jaccard_similarity(top_k_sigma, top_k_R)
                
                for f in f_values:
                    f_count = max(1, int(f * num_nodes))
                    top_f_nodes = [node for node, _ in ranked_nodes][:f_count]
                    si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * num_nodes)
                    rbo_scores[f'RBO {f:.2f}'] = compute_rbo(sigma[:f_count], R[:f_count])
                
                results[(method_name, beta)] = {
                    'Beta': beta,
                    'Kendall Tau': tau,
                    'P-Value': p_value,
                    **jaccard_scores,
                    **si_scores,
                    **rbo_scores,
                    **mrr_scores
                }
        
        data = []
        for (method_name, beta), result in results.items():
            row = [
                method_name,
                method_results[method_name]['execution_time'],
                method_results[method_name].get('monotonicity', 'N/A'),
                result.get('Beta'),
                result.get('Kendall Tau'),
                result.get('P-Value')
            ] + [result.get(f'Jaccard k={int(k_factor * num_nodes)}') for k_factor in k_factors] + \
              [result.get(f'SI {f:.2f}') for f in f_values] + \
              [result.get(f'RBO {f:.2f}') for f in f_values] + \
              [result.get(f'MRR {k_factor:.2f}') for k_factor in k_factors]
            
            data.append(row)
        
        expected_columns = ['Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'Jaccard k={int(k_factor * num_nodes)}' for k_factor in k_factors] + \
                           [f'SI {f:.2f}' for f in f_values] + \
                           [f'RBO {f:.2f}' for f in f_values] + \
                           [f'MRR {k_factor:.2f}' for k_factor in k_factors]
        
        result_df = pd.DataFrame(data, columns=expected_columns)
        
        dataset_name = os.path.splitext(dataset_filename)[0]
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)
            df_sir.to_excel(writer, sheet_name='SIR')
            result_df.to_excel(writer, sheet_name='Results', index=False)
        
        print(f"Processed {dataset_filename}, results saved in {output_file}")


Processed Contiguous(SIR).xlsx, results saved in Contiguous(SIR)_results.xlsx
